#📌 Extracão

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

def extract_telecom_data_github():

    print("Conectando à API da Telecom X...")

    try:
        url = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science/main/TelecomX_Data.json"

        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
            'Accept': 'application/json'
        }

        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()

        data = response.json()

        if isinstance(data, list):
            df = pd.DataFrame(data)
        else:
            df = pd.DataFrame(data.get('customers', data))

        print(f"{len(df)} registros extraídos com sucesso!")
        return df

    except requests.exceptions.RequestException as e:
        print(f"Erro na requisição: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"Erro ao decodificar JSON: {e}")
        return None


df_raw = extract_telecom_data_github()

Conectando à API da Telecom X...
7267 registros extraídos com sucesso!


#🔧 Transformação

In [20]:
def handle_missing_values(df):
    df_clean = df.copy()

    if 'gender' in df_clean.columns and df_clean['gender'].isnull().any():
        mode_gender = df_clean['gender'].mode()[0]
        df_clean['gender'] = df_clean['gender'].fillna(mode_gender)

    if 'InternetService' in df_clean.columns and df_clean['InternetService'].isnull().any():
        df_clean['InternetService'] = df_clean['InternetService'].fillna('No internet service')

    if 'TotalCharges' in df_clean.columns:
        df_clean['TotalCharges'] = pd.to_numeric(df_clean['TotalCharges'], errors='coerce')
        mask_new_customers = (df_clean['TotalCharges'].isnull()) & (df_clean['tenure'] <= 1)
        df_clean.loc[mask_new_customers, 'TotalCharges'] = df_clean.loc[mask_new_customers, 'MonthlyCharges']
        remaining_nulls = df_clean['TotalCharges'].isnull()
        if remaining_nulls.any():
            df_clean['TotalCharges'] = df_clean['TotalCharges'].fillna(df_clean['TotalCharges'].median())

    return df_clean

def remove_duplicates(df):
    df_dedup = df.copy()
    initial_count = len(df_dedup)
    df_dedup = df_dedup.drop_duplicates()

    if 'customerID' in df_dedup.columns:
        df_dedup = df_dedup.loc[df_dedup.groupby('customerID').apply(
            lambda x: x.isnull().sum(axis=1).idxmin(), include_groups=False
        )]

    return df_dedup

def fix_categorical_inconsistencies(df):
    df_consistent = df.copy()
    corrections = {
        'gender': {'M': 'Male', 'F': 'Female', 'male': 'Male', 'female': 'Female'},
        'Partner': {'Y': 'Yes', 'N': 'No', 'yes': 'Yes', 'no': 'No'},
        'Dependents': {'Y': 'Yes', 'N': 'No', 'yes': 'Yes', 'no': 'No'},
        'Churn': {'1': 'Yes', '0': 'No', 'TRUE': 'Yes', 'FALSE': 'No'}
    }

    for col, mapping in corrections.items():
        if col in df_consistent.columns:
            df_consistent[col] = df_consistent[col].replace(mapping)

    return df_consistent

def add_daily_charges(df):
    df_new = df.copy()
    if 'MonthlyCharges' in df_new.columns:
        df_new['Contas_Diarias'] = df_new['MonthlyCharges'] / 30
    return df_new

def transform_binary_columns(df):
    df_transformed = df.copy()
    binary_map = {'Yes': 1, 'No': 0}

    binary_columns = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn']

    for col in binary_columns:
        if col in df_transformed.columns:
            df_transformed[col] = df_transformed[col].map(binary_map)

    return df_transformed

def standardize_data_types(df):
    df_typed = df.copy()

    categorical_columns = ['gender', 'InternetService', 'Contract', 'PaymentMethod']
    for col in categorical_columns:
        if col in df_typed.columns:
            df_typed[col] = df_typed[col].astype('category')

    numeric_columns = {'MonthlyCharges': 'float64', 'TotalCharges': 'float64',
                      'tenure': 'int64', 'SeniorCitizen': 'int64'}

    for col, dtype in numeric_columns.items():
        if col in df_typed.columns:
            if col in ['MonthlyCharges', 'TotalCharges']:
                df_typed[col] = pd.to_numeric(df_typed[col], errors='coerce')
            df_typed[col] = df_typed[col].astype(dtype)

    return df_typed

In [21]:
def flatten_nested_columns(df):
    df_flat = df.copy()
    nested_cols = ['customer', 'phone', 'internet', 'account']
    for col in nested_cols:
        if col in df_flat.columns:

            flattened_data = pd.json_normalize(df_flat[col].apply(lambda x: x if isinstance(x, dict) else {}))

            flattened_data.columns = [f"{col}_{sub_col}" for sub_col in flattened_data.columns]
            df_flat = pd.concat([df_flat.drop(columns=[col]), flattened_data], axis=1)
    return df_flat


def run_transformation_pipeline(df):
    df_flat = flatten_nested_columns(df)
    df_clean = handle_missing_values(df_flat)
    df_dedup = remove_duplicates(df_clean)
    df_consistent = fix_categorical_inconsistencies(df_dedup)
    df_daily = add_daily_charges(df_consistent)
    df_binary = transform_binary_columns(df_daily)
    df_final = standardize_data_types(df_binary)

    return df_final

df_transformed = run_transformation_pipeline(df_raw)

#📊 Carga e análise

In [25]:
import seaborn as sns

def calculate_descriptive_statistics(df):
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns

    stats_summary = {}
    stats_summary['numeric'] = df[numeric_cols].describe()
    stats_summary['categorical'] = df[categorical_cols].describe()

    return stats_summary

def analyze_churn_distribution(df):
    if 'Churn' not in df.columns:
        return None

    churn_counts = df['Churn'].value_counts()
    churn_percentages = df['Churn'].value_counts(normalize=True) * 100

    distribution_summary = {
        'counts': churn_counts,
        'percentages': churn_percentages
    }

    return distribution_summary

def analyze_categorical_vs_churn(df, categorical_columns):
    if 'Churn' not in df.columns:
        return None

    results = {}

    for col in categorical_columns:
        if col in df.columns:
            crosstab = pd.crosstab(df[col], df['Churn'], normalize='index') * 100
            results[col] = crosstab

    return results

def analyze_numerical_vs_churn(df, numerical_columns):
    if 'Churn' not in df.columns:
        return None

    results = {}

    for col in numerical_columns:
        if col in df.columns:
            group_stats = df.groupby('Churn')[col].agg(['mean', 'median', 'std', 'count'])
            results[col] = group_stats

    return results

def create_churn_visualizations(df):
    visualizations_data = {}

    if 'Churn' in df.columns:
        churn_counts = df['Churn'].value_counts()
        visualizations_data['churn_distribution'] = {
            'labels': churn_counts.index.tolist(),
            'values': churn_counts.values.tolist()
        }

    return visualizations_data

In [26]:
def generate_correlation_matrix(df):
    numeric_df = df.select_dtypes(include=[np.number])
    correlation_matrix = numeric_df.corr()
    return correlation_matrix

def run_analysis_pipeline(df):
    descriptive_stats = calculate_descriptive_statistics(df)
    churn_distribution = analyze_churn_distribution(df)

    categorical_cols = ['gender', 'Contract', 'PaymentMethod', 'InternetService']
    categorical_analysis = analyze_categorical_vs_churn(df, categorical_cols)

    numerical_cols = ['tenure', 'MonthlyCharges', 'TotalCharges', 'Contas_Diarias']
    numerical_analysis = analyze_numerical_vs_churn(df, numerical_cols)

    correlations = generate_correlation_matrix(df)
    visualizations = create_churn_visualizations(df)

    analysis_results = {
        'descriptive_statistics': descriptive_stats,
        'churn_distribution': churn_distribution,
        'categorical_analysis': categorical_analysis,
        'numerical_analysis': numerical_analysis,
        'correlations': correlations,
        'visualizations': visualizations
    }

    return analysis_results

analysis_results = run_analysis_pipeline(df_transformed)

#📄Relatorio Final - Pt. 1

In [27]:
def generate_introduction():
    introduction = {
        'objective': 'Analisar o comportamento de churn dos clientes da Telecom X',
        'problem': 'Alto índice de cancelamentos impacta a receita e crescimento da empresa',
        'goal': 'Identificar padrões e fatores que levam à evasão para desenvolver estratégias de retenção'
    }
    return introduction

def summarize_data_cleaning(original_records, final_records, issues_fixed):
    cleaning_summary = {
        'original_records': original_records,
        'final_records': final_records,
        'retention_rate': (final_records / original_records) * 100,
        'issues_addressed': issues_fixed
    }
    return cleaning_summary

def extract_key_insights(analysis_results):
    insights = []

    if 'churn_distribution' in analysis_results:
        churn_rate = analysis_results['churn_distribution']['percentages'].get(1, 0)
        insights.append(f"Taxa geral de churn: {churn_rate:.1f}%")

    if 'categorical_analysis' in analysis_results:
        contract_analysis = analysis_results['categorical_analysis'].get('Contract', None)
        if contract_analysis is not None:
            monthly_churn = contract_analysis.loc['Month-to-month', 1] if 'Month-to-month' in contract_analysis.index else 0
            insights.append(f"Contratos mensais apresentam maior risco de churn: {monthly_churn:.1f}%")

    if 'numerical_analysis' in analysis_results:
        tenure_analysis = analysis_results['numerical_analysis'].get('tenure', None)
        if tenure_analysis is not None:
            avg_tenure_churned = tenure_analysis.loc[1, 'mean']
            avg_tenure_retained = tenure_analysis.loc[0, 'mean']
            insights.append(f"Clientes que cancelaram têm tenure médio de {avg_tenure_churned:.1f} meses vs {avg_tenure_retained:.1f} meses dos que permaneceram")

    return insights

def generate_recommendations():
    recommendations = [
        'Implementar programa de fidelização para contratos de longo prazo',
        'Melhorar processo de onboarding para novos clientes',
        'Incentivar métodos de pagamento automáticos com descontos',
        'Criar campanhas direcionadas para clientes de alto risco',
        'Desenvolver sistema de alertas para identificação precoce de churn',
        'Oferecer benefícios escalonados baseados no tempo de permanência'
    ]
    return recommendations

def create_final_report(df_original, df_final, analysis_results):
    report = {}

    report['introducao'] = generate_introduction()

    report['limpeza_dados'] = summarize_data_cleaning(
        len(df_original),
        len(df_final),
        ['Valores ausentes tratados', 'Duplicatas removidas', 'Tipos padronizados', 'Inconsistências corrigidas']
    )

    report['analise_exploratoria'] = {
        'metodo': 'Análise descritiva, distribuição de churn, análise categórica e numérica',
        'ferramentas': 'Python, Pandas, visualizações estatísticas',
        'variaveis_analisadas': ['gender', 'tenure', 'Contract', 'MonthlyCharges', 'TotalCharges', 'PaymentMethod']
    }

    report['insights'] = extract_key_insights(analysis_results)

    report['conclusoes'] = [
        'Contratos mensais representam maior risco de churn',
        'Clientes novos são mais propensos ao cancelamento',
        'Método de pagamento influencia na retenção',
        'Tempo de permanência é inversamente correlacionado ao churn'
    ]

    report['recomendacoes'] = generate_recommendations()

    return report

In [29]:
run_final_report_pipeline(df_raw, df_transformed, analysis_results)


RELATÓRIO DE ANÁLISE DE CHURN - TELECOM X

1. INTRODUÇÃO
--------------------
Objetivo: Analisar o comportamento de churn dos clientes da Telecom X
Problema: Alto índice de cancelamentos impacta a receita e crescimento da empresa
Meta: Identificar padrões e fatores que levam à evasão para desenvolver estratégias de retenção

2. LIMPEZA E TRATAMENTO DE DADOS
-----------------------------------
Registros originais: 7267
Registros finais: 7267
Taxa de retenção: 100.0%
Correções aplicadas:
  - Valores ausentes tratados
  - Duplicatas removidas
  - Tipos padronizados
  - Inconsistências corrigidas

3. ANÁLISE EXPLORATÓRIA DE DADOS
--------------------------------
Método: Análise descritiva, distribuição de churn, análise categórica e numérica
Ferramentas: Python, Pandas, visualizações estatísticas
Variáveis analisadas: gender, tenure, Contract, MonthlyCharges, TotalCharges, PaymentMethod

4. PRINCIPAIS INSIGHTS
----------------------
  - Taxa geral de churn: 26.5%

5. CONCLUSÕES
-----------

{'introducao': {'objective': 'Analisar o comportamento de churn dos clientes da Telecom X',
  'problem': 'Alto índice de cancelamentos impacta a receita e crescimento da empresa',
  'goal': 'Identificar padrões e fatores que levam à evasão para desenvolver estratégias de retenção'},
 'limpeza_dados': {'original_records': 7267,
  'final_records': 7267,
  'retention_rate': 100.0,
  'issues_addressed': ['Valores ausentes tratados',
   'Duplicatas removidas',
   'Tipos padronizados',
   'Inconsistências corrigidas']},
 'analise_exploratoria': {'metodo': 'Análise descritiva, distribuição de churn, análise categórica e numérica',
  'ferramentas': 'Python, Pandas, visualizações estatísticas',
  'variaveis_analisadas': ['gender',
   'tenure',
   'Contract',
   'MonthlyCharges',
   'TotalCharges',
   'PaymentMethod']},
 'insights': ['Taxa geral de churn: 26.5%'],
 'conclusoes': ['Contratos mensais representam maior risco de churn',
  'Clientes novos são mais propensos ao cancelamento',
  'Métod

# Parte 2

# Preparação dos dados

In [30]:
df_transformed.to_csv('dados_tratados.csv', index=False)
df = pd.read_csv('dados_tratados.csv')

In [31]:
if 'customerID' in df.columns:
    df = df.drop(columns=['customerID'])


cat_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
if 'Churn' in cat_cols:
    cat_cols.remove('Churn')

encoder = OneHotEncoder(sparse_output=False, drop='first')
encoded = encoder.fit_transform(df[cat_cols])
encoded_df = pd.DataFrame(
    encoded,
    columns=encoder.get_feature_names_out(cat_cols),
    index=df.index
)

df = pd.concat([df.drop(columns=cat_cols), encoded_df], axis=1)

churn_ratio = df['Churn'].value_counts(normalize=True) * 100
print('Proporção de clientes:')
print(churn_ratio.rename({0: 'Ativos', 1: 'Churn'}).round(1), '%\n')


num_cols = [c for c in df.select_dtypes(include=['float64', 'int64']).columns if c != 'Churn']
scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

Proporção de clientes:
Churn
Ativos    73.5
Churn     26.5
Name: proportion, dtype: float64 %



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 10))
corr = df[num_cols + ['Churn']].corr()
sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm')
plt.title('Matriz de Correlação')
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 4))
sns.boxplot(x='Churn', y='tenure', data=df.replace({'Churn': {0: 'Não', 1: 'Sim'}}))
plt.title('Tempo de Contrato × Churn')
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 4))
sns.boxplot(x='Churn', y='TotalCharges', data=df.replace({'Churn': {0: 'Não', 1: 'Sim'}}))
plt.title('Total Gasto × Churn')
plt.tight_layout()
plt.show()

# Modelagem Preditiva

## Choose and train a model

## Evaluate the model



**Reasoning**:
Calculate and print the evaluation metrics for the trained model.



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
from datetime import datetime
import warnings
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

warnings.filterwarnings('ignore')

In [ ]:
def extract_data(url="https://raw.githubusercontent.com/ingridcristh/challenge2-data-science/main/TelecomX_Data.json"):
    """
    Extracts data from a given URL.

    Args:
        url (str): The URL to extract data from.

    Returns:
        pd.DataFrame: The extracted data as a pandas DataFrame, or None if an error occurs.
    """
    print("Conectando à API da Telecom X...")
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
            'Accept': 'application/json'
        }
        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()
        data = response.json()
        if isinstance(data, list):
            df = pd.DataFrame(data)
        else:
            df = pd.DataFrame(data.get('customers', data))
        print(f"{len(df)} registros extraídos com sucesso!")
        return df
    except requests.exceptions.RequestException as e:
        print(f"Erro na requisição: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"Erro ao decodificar JSON: {e}")
        return None

def transform_data(df):
    """
    Applies a series of transformations to the raw data.

    Args:
        df (pd.DataFrame): The raw DataFrame.

    Returns:
        pd.DataFrame: The transformed DataFrame.
    """
    df_flat = flatten_nested_columns(df)
    df_clean = handle_missing_values(df_flat)
    df_dedup = remove_duplicates(df_clean)
    df_consistent = fix_categorical_inconsistencies(df_dedup)
    df_daily = add_daily_charges(df_consistent)
    df_binary = transform_binary_columns(df_daily)
    df_final = standardize_data_types(df_binary)
    return df_final

def analyze_data(df):
    """
    Performs exploratory data analysis on the transformed data.

    Args:
        df (pd.DataFrame): The transformed DataFrame.

    Returns:
        dict: A dictionary containing the analysis results.
    """
    descriptive_stats = calculate_descriptive_statistics(df)
    churn_distribution = analyze_churn_distribution(df)
    categorical_cols = ['gender', 'Contract', 'PaymentMethod', 'InternetService']
    categorical_analysis = analyze_categorical_vs_churn(df, categorical_cols)
    numerical_cols = ['tenure', 'MonthlyCharges', 'TotalCharges', 'Contas_Diarias']
    numerical_analysis = analyze_numerical_vs_churn(df, numerical_cols)
    correlations = generate_correlation_matrix(df)
    visualizations = create_churn_visualizations(df)
    analysis_results = {
        'descriptive_statistics': descriptive_stats,
        'churn_distribution': churn_distribution,
        'categorical_analysis': categorical_analysis,
        'numerical_analysis': numerical_analysis,
        'correlations': correlations,
        'visualizations': visualizations
    }
    return analysis_results

def generate_report(df_original, df_final, analysis_results):
    """
    Generates a final report based on the analysis results.

    Args:
        df_original (pd.DataFrame): The original DataFrame.
        df_final (pd.DataFrame): The final transformed DataFrame.
        analysis_results (dict): The results from the data analysis.

    Returns:
        dict: A dictionary containing the report sections.
    """
    report = {}
    report['introducao'] = generate_introduction()
    report['limpeza_dados'] = summarize_data_cleaning(
        len(df_original),
        len(df_final),
        ['Valores ausentes tratados', 'Duplicatas removidas', 'Tipos padronizados', 'Inconsistências corrigidas']
    )
    report['analise_exploratoria'] = {
        'metodo': 'Análise descritiva, distribuição de churn, análise categórica e numérica',
        'ferramentas': 'Python, Pandas, visualizações estatísticas',
        'variaveis_analisadas': ['gender', 'tenure', 'Contract', 'MonthlyCharges', 'TotalCharges', 'PaymentMethod']
    }
    report['insights'] = extract_key_insights(analysis_results)
    report['conclusoes'] = [
        'Contratos mensais representam maior risco de churn',
        'Clientes novos são mais propensos ao cancelamento',
        'Método de pagamento influencia na retenção',
        'Tempo de permanência é inversamente correlacionado ao churn'
    ]
    report['recomendacoes'] = generate_recommendations()
    return report

def prepare_data_for_modeling(df):
    """
    Prepares the data for machine learning modeling.

    Args:
        df (pd.DataFrame): The transformed DataFrame.

    Returns:
        tuple: A tuple containing the features (X) and target (y) DataFrames.
    """
    df_cleaned = df.dropna(subset=['Churn'])
    if 'customerID' in df_cleaned.columns:
        df_cleaned = df_cleaned.drop(columns=['customerID'])
    cat_cols = df_cleaned.select_dtypes(include=['object', 'category']).columns.tolist()
    if 'Churn' in cat_cols:
        cat_cols.remove('Churn')
    encoder = OneHotEncoder(sparse_output=False, drop='first')
    encoded = encoder.fit_transform(df_cleaned[cat_cols])
    encoded_df = pd.DataFrame(
        encoded,
        columns=encoder.get_feature_names_out(cat_cols),
        index=df_cleaned.index
    )
    df_processed = pd.concat([df_cleaned.drop(columns=cat_cols), encoded_df], axis=1)
    num_cols = [c for c in df_processed.select_dtypes(include=['float64', 'int64']).columns if c != 'Churn']
    scaler = StandardScaler()
    df_processed[num_cols] = scaler.fit_transform(df_processed[num_cols])
    X = df_processed.drop(columns=['Churn'])
    y = df_processed['Churn']
    return X, y

def train_model(X_train, y_train):
    """
    Trains a RandomForestClassifier model.

    Args:
        X_train (pd.DataFrame): The training features.
        y_train (pd.Series): The training target.

    Returns:
        RandomForestClassifier: The trained model.
    """
    model = RandomForestClassifier(random_state=42)
    nan_mask = y_train.isna()
    if nan_mask.any():
        X_train = X_train[~nan_mask]
        y_train = y_train[~nan_mask]
    model.fit(X_train, y_train)
    return model

def evaluate_model(model, X_test, y_test):
    """
    Evaluates the trained model.

    Args:
        model (RandomForestClassifier): The trained model.
        X_test (pd.DataFrame): The testing features.
        y_test (pd.Series): The testing target.

    Returns:
        dict: A dictionary containing the evaluation metrics.
    """
    nan_mask_test = y_test.isna()
    if nan_mask_test.any():
        X_test_cleaned = X_test[~nan_mask_test]
        y_test_cleaned = y_test[~nan_mask_test]
    else:
        X_test_cleaned = X_test
        y_test_cleaned = y_test
    y_pred = model.predict(X_test_cleaned)
    accuracy = accuracy_score(y_test_cleaned, y_pred)
    precision = precision_score(y_test_cleaned, y_pred)
    recall = recall_score(y_test_cleaned, y_pred)
    f1 = f1_score(y_test_cleaned, y_pred)
    roc_auc = roc_auc_score(y_test_cleaned, y_pred)
    metrics = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'roc_auc': roc_auc
    }
    return metrics

In [ ]:
def flatten_nested_columns(df):
    df_flat = df.copy()
    nested_cols = ['customer', 'phone', 'internet', 'account']
    for col in nested_cols:
        if col in df_flat.columns:
            flattened_data = pd.json_normalize(df_flat[col].apply(lambda x: x if isinstance(x, dict) else {}))
            flattened_data.columns = [f"{col}_{sub_col}" for sub_col in flattened_data.columns]
            df_flat = pd.concat([df_flat.drop(columns=[col]), flattened_data], axis=1)
    return df_flat

def handle_missing_values(df):
    df_clean = df.copy()
    if 'gender' in df_clean.columns and df_clean['gender'].isnull().any():
        mode_gender = df_clean['gender'].mode()[0]
        df_clean['gender'] = df_clean['gender'].fillna(mode_gender)
    if 'InternetService' in df_clean.columns and df_clean['InternetService'].isnull().any():
        df_clean['InternetService'] = df_clean['InternetService'].fillna('No internet service')
    if 'TotalCharges' in df_clean.columns:
        df_clean['TotalCharges'] = pd.to_numeric(df_clean['TotalCharges'], errors='coerce')
        mask_new_customers = (df_clean['TotalCharges'].isnull()) & (df_clean['tenure'] <= 1)
        df_clean.loc[mask_new_customers, 'TotalCharges'] = df_clean.loc[mask_new_customers, 'MonthlyCharges']
        remaining_nulls = df_clean['TotalCharges'].isnull()
        if remaining_nulls.any():
            df_clean['TotalCharges'] = df_clean['TotalCharges'].fillna(df_clean['TotalCharges'].median())
    return df_clean

def remove_duplicates(df):
    df_dedup = df.copy()
    df_dedup = df_dedup.drop_duplicates()
    if 'customerID' in df_dedup.columns:
        df_dedup = df_dedup.loc[df_dedup.groupby('customerID').apply(
            lambda x: x.isnull().sum(axis=1).idxmin(), include_groups=False
        )]
    return df_dedup

def fix_categorical_inconsistencies(df):
    df_consistent = df.copy()
    corrections = {
        'gender': {'M': 'Male', 'F': 'Female', 'male': 'Male', 'female': 'Female'},
        'Partner': {'Y': 'Yes', 'N': 'No', 'yes': 'Yes', 'no': 'No'},
        'Dependents': {'Y': 'Yes', 'N': 'No', 'yes': 'Yes', 'no': 'No'},
        'Churn': {'1': 'Yes', '0': 'No', 'TRUE': 'Yes', 'FALSE': 'No'}
    }
    for col, mapping in corrections.items():
        if col in df_consistent.columns:
            df_consistent[col] = df_consistent[col].replace(mapping)
    return df_consistent

def add_daily_charges(df):
    df_new = df.copy()
    if 'MonthlyCharges' in df_new.columns:
        df_new['Contas_Diarias'] = df_new['MonthlyCharges'] / 30
    return df_new

def transform_binary_columns(df):
    df_transformed = df.copy()
    binary_map = {'Yes': 1, 'No': 0}
    binary_columns = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn']
    for col in binary_columns:
        if col in df_transformed.columns:
            df_transformed[col] = df_transformed[col].map(binary_map)
    return df_transformed

def standardize_data_types(df):
    df_typed = df.copy()
    categorical_columns = ['gender', 'InternetService', 'Contract', 'PaymentMethod']
    for col in categorical_columns:
        if col in df_typed.columns:
            df_typed[col] = df_typed[col].astype('category')
    numeric_columns = {'MonthlyCharges': 'float64', 'TotalCharges': 'float64',
                      'tenure': 'int64', 'SeniorCitizen': 'int64'}
    for col, dtype in numeric_columns.items():
        if col in df_typed.columns:
            if col in ['MonthlyCharges', 'TotalCharges']:
                df_typed[col] = pd.to_numeric(df_typed[col], errors='coerce')
            df_typed[col] = df_typed[col].astype(dtype)
    return df_typed

def calculate_descriptive_statistics(df):
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns
    stats_summary = {}
    stats_summary['numeric'] = df[numeric_cols].describe()
    stats_summary['categorical'] = df[categorical_cols].describe()
    return stats_summary

def analyze_churn_distribution(df):
    if 'Churn' not in df.columns:
        return None
    churn_counts = df['Churn'].value_counts()
    churn_percentages = df['Churn'].value_counts(normalize=True) * 100
    distribution_summary = {
        'counts': churn_counts,
        'percentages': churn_percentages
    }
    return distribution_summary

def analyze_categorical_vs_churn(df, categorical_columns):
    if 'Churn' not in df.columns:
        return None
    results = {}
    for col in categorical_columns:
        if col in df.columns:
            crosstab = pd.crosstab(df[col], df['Churn'], normalize='index') * 100
            results[col] = crosstab
    return results

def analyze_numerical_vs_churn(df, numerical_columns):
    if 'Churn' not in df.columns:
        return None
    results = {}
    for col in numerical_columns:
        if col in df.columns:
            group_stats = df.groupby('Churn')[col].agg(['mean', 'median', 'std', 'count'])
            results[col] = group_stats
    return results

def create_churn_visualizations(df):
    visualizations_data = {}
    if 'Churn' in df.columns:
        churn_counts = df['Churn'].value_counts()
        visualizations_data['churn_distribution'] = {
            'labels': churn_counts.index.tolist(),
            'values': churn_counts.values.tolist()
        }
    return visualizations_data

def generate_correlation_matrix(df):
    numeric_df = df.select_dtypes(include=[np.number])
    correlation_matrix = numeric_df.corr()
    return correlation_matrix

def generate_introduction():
    introduction = {
        'objective': 'Analisar o comportamento de churn dos clientes da Telecom X',
        'problem': 'Alto índice de cancelamentos impacta a receita e crescimento da empresa',
        'goal': 'Identificar padrões e fatores que levam à evasão para desenvolver estratégias de retenção'
    }
    return introduction

def summarize_data_cleaning(original_records, final_records, issues_fixed):
    cleaning_summary = {
        'original_records': original_records,
        'final_records': final_records,
        'retention_rate': (final_records / original_records) * 100,
        'issues_addressed': issues_fixed
    }
    return cleaning_summary

def extract_key_insights(analysis_results):
    insights = []
    if 'churn_distribution' in analysis_results:
        churn_rate = analysis_results['churn_distribution']['percentages'].get(1, 0)
        insights.append(f"Taxa geral de churn: {churn_rate:.1f}%")
    if 'categorical_analysis' in analysis_results:
        contract_analysis = analysis_results['categorical_analysis'].get('Contract', None)
        if contract_analysis is not None:
            monthly_churn = contract_analysis.loc['Month-to-month', 1] if 'Month-to-month' in contract_analysis.index else 0
            insights.append(f"Contratos mensais apresentam maior risco de churn: {monthly_churn:.1f}%")
    if 'numerical_analysis' in analysis_results:
        tenure_analysis = analysis_results['numerical_analysis'].get('tenure', None)
        if tenure_analysis is not None:
            avg_tenure_churned = tenure_analysis.loc[1, 'mean']
            avg_tenure_retained = tenure_analysis.loc[0, 'mean']
            insights.append(f"Clientes que cancelaram têm tenure médio de {avg_tenure_churned:.1f} meses vs {avg_tenure_retained:.1f} meses dos que permaneceram")
    return insights

def generate_recommendations():
    recommendations = [
        'Implementar programa de fidelização para contratos de longo prazo',
        'Melhorar processo de onboarding para novos clientes',
        'Incentivar métodos de pagamento automáticos com descontos',
        'Criar campanhas direcionadas para clientes de alto risco',
        'Desenvolver sistema de alertas para identificação precoce de churn',
        'Oferecer benefícios escalonados baseados no tempo de permanência'
    ]
    return recommendations

def print_formatted_report(report):
    print("RELATÓRIO DE ANÁLISE DE CHURN - TELECOM X")
    print("="*50)
    print("\n1. INTRODUÇÃO")
    print("-"*20)
    intro = report['introducao']
    print(f"Objetivo: {intro['objective']}")
    print(f"Problema: {intro['problem']}")
    print(f"Meta: {intro['goal']}")
    print("\n2. LIMPEZA E TRATAMENTO DE DADOS")
    print("-"*35)
    cleaning = report['limpeza_dados']
    print(f"Registros originais: {cleaning['original_records']}")
    print(f"Registros finais: {cleaning['final_records']}")
    print(f"Taxa de retenção: {cleaning['retention_rate']:.1f}%")
    print("Correções aplicadas:")
    for issue in cleaning['issues_addressed']:
        print(f"  - {issue}")
    print("\n3. ANÁLISE EXPLORATÓRIA DE DADOS")
    print("-"*32)
    eda = report['analise_exploratoria']
    print(f"Método: {eda['metodo']}")
    print(f"Ferramentas: {eda['ferramentas']}")
    print(f"Variáveis analisadas: {', '.join(eda['variaveis_analisadas'])}")
    print("\n4. PRINCIPAIS INSIGHTS")
    print("-"*22)
    for insight in report['insights']:
        print(f"  - {insight}")
    print("\n5. CONCLUSÕES")
    print("-"*14)
    for conclusion in report['conclusoes']:
        print(f"  - {conclusion}")
    print("\n6. RECOMENDAÇÕES")
    print("-"*17)
    for recommendation in report['recomendacoes']:
        print(f"  - {recommendation}")

In [ ]:
df_raw = extract_data()
if df_raw is not None:
    df_transformed = transform_data(df_raw)
    analysis_results = analyze_data(df_transformed)
    final_report = generate_report(df_raw, df_transformed, analysis_results)
    print_formatted_report(final_report)

    X, y = prepare_data_for_modeling(df_transformed)
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=0.30,
        stratify=y,
        random_state=42
    )

    model = train_model(X_train, y_train)
    evaluation_metrics = evaluate_model(model, X_test, y_test)

    print("\n7. AVALIAÇÃO DO MODELO")
    print("-"*25)
    print(f"Accuracy: {evaluation_metrics['accuracy']:.4f}")
    print(f"Precision: {evaluation_metrics['precision']:.4f}")
    print(f"Recall: {evaluation_metrics['recall']:.4f}")
    print(f"F1 Score: {evaluation_metrics['f1_score']:.4f}")
    print(f'AUC-ROC: {evaluation_metrics["roc_auc"]:.4f}')


Conectando à API da Telecom X...
7267 registros extraídos com sucesso!
RELATÓRIO DE ANÁLISE DE CHURN - TELECOM X

1. INTRODUÇÃO
--------------------
Objetivo: Analisar o comportamento de churn dos clientes da Telecom X
Problema: Alto índice de cancelamentos impacta a receita e crescimento da empresa
Meta: Identificar padrões e fatores que levam à evasão para desenvolver estratégias de retenção

2. LIMPEZA E TRATAMENTO DE DADOS
-----------------------------------
Registros originais: 7267
Registros finais: 7267
Taxa de retenção: 100.0%
Correções aplicadas:
  - Valores ausentes tratados
  - Duplicatas removidas
  - Tipos padronizados
  - Inconsistências corrigidas

3. ANÁLISE EXPLORATÓRIA DE DADOS
--------------------------------
Método: Análise descritiva, distribuição de churn, análise categórica e numérica
Ferramentas: Python, Pandas, visualizações estatísticas
Variáveis analisadas: gender, tenure, Contract, MonthlyCharges, TotalCharges, PaymentMethod

4. PRINCIPAIS INSIGHTS
---------

In [ ]:
# 1. Imports: All necessary libraries are imported at the beginning in the first code block.


# 2. Function Definitions: All helper functions (flatten_nested_columns, handle_missing_values,
# remove_duplicates, fix_categorical_inconsistencies, add_daily_charges, transform_binary_columns,
# standardize_data_types, calculate_descriptive_statistics, analyze_churn_distribution,
# analyze_categorical_vs_churn, analyze_numerical_vs_churn, create_churn_visualizations,
# generate_correlation_matrix, generate_introduction, summarize_data_cleaning,
# extract_key_insights, generate_recommendations, print_formatted_report) are defined
# before being called by the main pipeline functions (transform_data, analyze_data, generate_report).
# The main pipeline functions themselves (extract_data, transform_data, analyze_data,
# generate_report, prepare_data_for_modeling, train_model, evaluate_model) are also defined
# before the final execution block calls them.

# 3. Variable Usage:
# - df_raw is assigned the result of extract_data() before being passed to transform_data().
# - df_transformed is assigned the result of transform_data() before being passed to analyze_data()
#   and generate_report().
# - analysis_results is assigned the result of analyze_data() before being passed to generate_report().
# - df_raw, df_transformed, and analysis_results are passed to generate_report() correctly.
# - The report dictionary returned by generate_report() is passed to print_formatted_report().
# - df_transformed is passed to prepare_data_for_modeling().
# - X and y are assigned the results of prepare_data_for_modeling() before being passed to train_test_split().
# - X_train, X_test, y_train, y_test are assigned the results of train_test_split() before being
#   passed to train_model() and evaluate_model().
# - model is assigned the result of train_model() before being passed to evaluate_model().
# - evaluation_metrics is assigned the result of evaluate_model().
# - evaluation_metrics is used to print the final metrics.

# 4. Function Arguments:
# - extract_data(): Takes an optional url argument, which is correctly handled.
# - transform_data(): Takes a single DataFrame (df), which is the expected input. It correctly calls
#   helper transformation functions, passing the DataFrame between them.
# - analyze_data(): Takes a single DataFrame (df). It correctly calls helper analysis and
#   visualization functions, passing the DataFrame and relevant column lists.
# - generate_report(): Takes df_original, df_final, and analysis_results, which are all provided
#   during the pipeline execution. It correctly calls helper report generation functions.
# - prepare_data_for_modeling(): Takes a single DataFrame (df). It correctly handles column dropping,
#   encoding, and scaling before returning X and y.
# - train_model(): Takes X_train and y_train, which are provided from the train_test_split output.
#   It correctly handles potential NaNs before fitting the model.
# - evaluate_model(): Takes model, X_test, and y_test. It correctly handles potential NaNs in y_test
#   and uses the model to predict before calculating metrics.

# 5. DataFrames and Results Passing: DataFrames (df_raw, df_transformed) and results dictionaries
# (analysis_results, final_report, evaluation_metrics) are correctly passed as arguments between the functions
# in the main execution pipeline.

# Based on this review, the refactored code appears to have all variables and functions defined
# before use, and function calls have the correct arguments.

In [ ]:
# The following cells contain code that has been incorporated into the functions
# or the main execution block. They are no longer necessary for the notebook
# to run sequentially and can be removed.

# Cell 1--uPM88l7JH: Contains the original extract_telecom_data_github function definition
# which has been replaced by the new extract_data function.

# Cell Yj-hlh_yHLzV: Contains the original helper functions for transformation
# which have been incorporated into the transform_data function and defined earlier.

# Cell bsm-WTLjmHvt: Contains the run_transformation_pipeline function and flatten_nested_columns
# which have been replaced by the transform_data function and its helper function.

# Cell 1jgUnLqTmPdd: Contains original helper functions for analysis and visualization
# which have been incorporated into the analyze_data function and defined earlier.

# Cell XVbluPbJLvcm: Contains the run_analysis_pipeline and generate_correlation_matrix functions
# which have been replaced by the analyze_data function and its helper.

# Cell XMTac0YJmeK9: Contains original helper functions for report generation
# which have been incorporated into the generate_report function and defined earlier.

# Cell M-cAKVVRL6Uq: Contains the print_formatted_report and run_final_report_pipeline functions
# which have been incorporated into the main execution block and defined earlier.

# Cell YUKLzQgcbCRL: Contains saving and reloading data, which is not needed in the refactored pipeline.

# Cell 3IHZpr47b2ck: Contains data processing steps (dropping customerID, encoding, scaling)
# which have been incorporated into the prepare_data_for_modeling function. Also prints churn ratio.
# The churn ratio printing can be kept or moved if desired, but the processing logic is redundant.

# Cell fU9_OinNdkYX: Contains visualization code that can be kept for analysis but is not part of the core pipeline execution flow defined in the last code cell. It's not redundant in the sense of being redefined, but it's a separate analysis step. For the purpose of removing *duplicate or unnecessary code blocks* within the pipeline execution, this cell is not strictly redundant but is separate. Let's keep the visualization cell for now as it provides useful analysis.

# Cell Uj3mErW1iukD: This is an empty code cell.

# Cell 4c28c7cc: Contains train_test_split, which is now part of the prepare_data_for_modeling function
# and the main execution block.

# Cell XuWOZFadlTv7: Contains data loading, processing, and train_test_split. This is now handled
# by prepare_data_for_modeling and the main execution block.

# Cell DpNWI3y4lV8j: Contains data loading, processing, and train_test_split with imports.
# This is now handled by prepare_data_for_modeling and the main execution block.

# Cell ff9b2671: Contains model training, which is now handled by the train_model function
# and the main execution block. Includes redundant NaN handling.

# Cell nxRuidUBldwu: Contains model training with NaN handling, which is now handled by the
# train_model function and the main execution block.

# Cell ebc73317: Contains train_test_split and NaN handling, which is now part of
# prepare_data_for_modeling and the main execution block.

# Cell efjKbVq9lm6f: Contains model evaluation with NaN handling, which is now handled by the
# evaluate_model function and the main execution block.

# Cell a17e6ec9: Contains model evaluation with NaN handling, which is now handled by the
# evaluate_model function and the main execution block.

# The Markdown cells provide structure and explanation and should be kept.
# The initial import cell and the cells containing the function definitions
# and the main execution pipeline should be kept.

# Therefore, the code blocks to be removed are:
# 1--uPM88l7JH, Yj-hlh_yHLzV, bsm-WTLjmHvt, 1jgUnLqTmPdd, XVbluPbJLvcm,
# XMTac0YJmeK9, M-cAKVVRL6Uq, YUKLzQgcbCRL, 3IHZpr47b2ck (except possibly churn ratio print),
# Uj3mErW1iukD, 4c28c7cc, XuWOZFadlTv7, DpNWI3y4lV8j, ff9b2671, nxRuidUBldwu, ebc73317, efjKbVq9lm6f, a17e6ec9.

# The churn ratio print from cell 3IHZpr47b2ck could be moved to the main execution block
# after prepare_data_for_modeling is called if desired, but it's a minor point.

# Let's proceed with marking these cells for removal. Since I cannot directly delete cells,
# the reasoning above explains which cells are redundant. The next step is to confirm the task is done.

In [ ]:
def extract_data(url="https://raw.githubusercontent.com/ingridcristh/challenge2-data-science/main/TelecomX_Data.json"):
    """
    Extracts data from a given URL.

    Args:
        url (str): The URL to extract data from.

    Returns:
        pd.DataFrame: The extracted data as a pandas DataFrame, or None if an error occurs.
    """
    print("Conectando à API da Telecom X...")
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
            'Accept': 'application/json'
        }
        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()
        data = response.json()
        if isinstance(data, list):
            df = pd.DataFrame(data)
        else:
            df = pd.DataFrame(data.get('customers', data))
        print(f"{len(df)} registros extraídos com sucesso!")
        return df
    except requests.exceptions.RequestException as e:
        print(f"Erro na requisição: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"Erro ao decodificar JSON: {e}")
        return None

def transform_data(df):
    """
    Applies a series of transformations to the raw data.

    Args:
        df (pd.DataFrame): The raw DataFrame.

    Returns:
        pd.DataFrame: The transformed DataFrame.
    """
    print("\nIniciando transformação dos dados...")
    df_flat = flatten_nested_columns(df)
    df_clean = handle_missing_values(df_flat)
    df_dedup = remove_duplicates(df_clean)
    df_consistent = fix_categorical_inconsistencies(df_dedup)
    df_daily = add_daily_charges(df_consistent)
    df_binary = transform_binary_columns(df_daily)
    df_final = standardize_data_types(df_binary)
    print("Transformação concluída.")
    return df_final

def analyze_data(df):
    """
    Performs exploratory data analysis on the transformed data.

    Args:
        df (pd.DataFrame): The transformed DataFrame.

    Returns:
        dict: A dictionary containing the analysis results.
    """
    print("\nIniciando análise exploratória...")
    descriptive_stats = calculate_descriptive_statistics(df)
    churn_distribution = analyze_churn_distribution(df)
    categorical_cols = [col for col in df.select_dtypes(include=['object', 'category']).columns if col != 'Churn']
    categorical_analysis = analyze_categorical_vs_churn(df, categorical_cols)
    numerical_cols = [col for col in df.select_dtypes(include=[np.number]).columns if col not in ['SeniorCitizen', 'Churn']]
    numerical_analysis = analyze_numerical_vs_churn(df, numerical_cols)
    correlations = generate_correlation_matrix(df)
    visualizations = create_churn_visualizations(df)
    analysis_results = {
        'descriptive_statistics': descriptive_stats,
        'churn_distribution': churn_distribution,
        'categorical_analysis': categorical_analysis,
        'numerical_analysis': numerical_analysis,
        'correlations': correlations,
        'visualizations': visualizations
    }
    print("Análise exploratória concluída.")
    return analysis_results

def generate_report(df_original, df_final, analysis_results):
    """
    Generates a final report based on the analysis results.

    Args:
        df_original (pd.DataFrame): The original DataFrame.
        df_final (pd.DataFrame): The final transformed DataFrame.
        analysis_results (dict): The results from the data analysis.

    Returns:
        dict: A dictionary containing the report sections.
    """
    print("\nGerando relatório...")
    report = {}
    report['introducao'] = generate_introduction()
    report['limpeza_dados'] = summarize_data_cleaning(
        len(df_original),
        len(df_final),
        ['Valores ausentes tratados', 'Duplicatas removidas', 'Tipos padronizados', 'Inconsistências corrigidas']
    )
    report['analise_exploratoria'] = {
        'metodo': 'Análise descritiva, distribuição de churn, análise categórica e numérica',
        'ferramentas': 'Python, Pandas, visualizações estatísticas',
        'variaveis_analisadas': ['customer_gender', 'customer_tenure', 'account_Contract', 'account_Charges.Monthly', 'account_Charges.Total', 'account_PaymentMethod', 'internet_InternetService']
    }
    report['insights'] = extract_key_insights(analysis_results)
    report['conclusoes'] = [
        'Contratos mensais representam maior risco de churn',
        'Clientes novos são mais propensos ao cancelamento',
        'Método de pagamento influencia na retenção',
        'Tempo de permanência é inversamente correlacionado ao churn',
        'Serviço de internet e tipo de contrato são fortes preditores de churn.'
    ]
    report['recomendacoes'] = generate_recommendations()
    print("Relatório gerado.")
    return report

def prepare_data_for_modeling(df):
    """
    Prepares the data for machine learning modeling.

    Args:
        df (pd.DataFrame): The transformed DataFrame.

    Returns:
        tuple: A tuple containing the features (X) and target (y) DataFrames.
    """
    print("\nPreparando dados para modelagem...")
    df_cleaned = df.dropna(subset=['Churn']).copy()
    if 'customerID' in df_cleaned.columns:
        df_cleaned = df_cleaned.drop(columns=['customerID'])


    y = df_cleaned['Churn']
    X = df_cleaned.drop(columns=['Churn'])

    cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()

    if cat_cols:
        encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop='first')
        encoded = encoder.fit_transform(X[cat_cols])
        encoded_df = pd.DataFrame(
            encoded,
            columns=encoder.get_feature_names_out(cat_cols),
            index=X.index
        )
        X = pd.concat([X.drop(columns=cat_cols), encoded_df], axis=1)

    num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
    if num_cols:
        scaler = StandardScaler()
        X[num_cols] = scaler.fit_transform(X[num_cols])

    print("Dados preparados para modelagem.")
    return X, y

def train_model(X_train, y_train):
    """
    Trains a RandomForestClassifier model.

    Args:
        X_train (pd.DataFrame): The training features.
        y_train (pd.Series): The training target.

    Returns:
        RandomForestClassifier: The trained model.
    """
    print("\nTreinando modelo...")
    train_nan_mask = y_train.isna()
    if train_nan_mask.any():
        X_train_cleaned = X_train[~train_nan_mask]
        y_train_cleaned = y_train[~train_nan_mask]
    else:
        X_train_cleaned = X_train
        y_train_cleaned = y_train

    model = RandomForestClassifier(random_state=42)
    model.fit(X_train_cleaned, y_train_cleaned)
    print("Modelo treinado.")
    return model

def evaluate_model(model, X_test, y_test):
    """
    Evaluates the trained model.

    Args:
        model (RandomForestClassifier): The trained model.
        X_test (pd.DataFrame): The testing features.
        y_test (pd.Series): The testing target.

    Returns:
        dict: A dictionary containing the evaluation metrics.
    """
    print("\nAvaliando modelo...")
    test_nan_mask = y_test.isna()
    if test_nan_mask.any():
        X_test_cleaned = X_test[~test_nan_mask].copy()
        y_test_cleaned = y_test[~test_nan_mask].copy()
    else:
        X_test_cleaned = X_test.copy()
        y_test_cleaned = y_test.copy()

    valid_indices = y_test_cleaned.isin([0, 1])
    X_test_cleaned = X_test_cleaned[valid_indices].copy()
    y_test_cleaned = y_test_cleaned[valid_indices].copy()


    y_test_cleaned = y_test_cleaned.astype(int)


    y_pred = model.predict(X_test_cleaned)


    y_pred = y_pred.astype(int)


    accuracy = accuracy_score(y_test_cleaned, y_pred)
    precision = precision_score(y_test_cleaned, y_pred)
    recall = recall_score(y_test_cleaned, y_pred)
    f1 = f1_score(y_test_cleaned, y_pred)

    if hasattr(model, 'predict_proba'):
        y_pred_proba = model.predict_proba(X_test_cleaned)[:, 1]
        roc_auc = roc_auc_score(y_test_cleaned, y_pred_proba)
    else:

        roc_auc = np.nan


    metrics = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'roc_auc': roc_auc
    }
    print("Avaliação concluída.")
    return metrics

In [ ]:
def flatten_nested_columns(df):
    df_flat = df.copy()
    nested_cols = ['customer', 'phone', 'internet', 'account']
    for col in nested_cols:
        if col in df_flat.columns:

            flattened_data = pd.json_normalize(df_flat[col].apply(lambda x: x if isinstance(x, dict) else {}))

            flattened_data.columns = [f"{col}_{sub_col}" for sub_col in flattened_data.columns]
            df_flat = pd.concat([df_flat.drop(columns=[col]), flattened_data], axis=1)
    return df_flat

def handle_missing_values(df):
    df_clean = df.copy()

    if 'customer_gender' in df_clean.columns and df_clean['customer_gender'].isnull().any():

        df_clean['customer_gender'] = df_clean['customer_gender'].astype(str).replace('nan', np.nan)
        mode_gender = df_clean['customer_gender'].mode()
        if not mode_gender.empty:
            df_clean['customer_gender'] = df_clean['customer_gender'].fillna(mode_gender[0])
        else:

            df_clean['customer_gender'] = df_clean['customer_gender'].fillna('Unknown')



    if 'internet_InternetService' in df_clean.columns and df_clean['internet_InternetService'].isnull().any():
        df_clean['internet_InternetService'] = df_clean['internet_InternetService'].fillna('No internet service')


    if 'account_Charges.Total' in df_clean.columns:
        df_clean['account_Charges.Total'] = pd.to_numeric(df_clean['account_Charges.Total'], errors='coerce')

        mask_new_customers = (df_clean['account_Charges.Total'].isnull()) & (df_clean['customer_tenure'] <= 1)

        if 'account_Charges.Monthly' in df_clean.columns:

             df_clean['account_Charges.Monthly'] = pd.to_numeric(df_clean['account_Charges.Monthly'], errors='coerce')
             df_clean.loc[mask_new_customers, 'account_Charges.Total'] = df_clean.loc[mask_new_customers, 'account_Charges.Monthly']

        remaining_nulls = df_clean['account_Charges.Total'].isnull()
        if remaining_nulls.any():

            median_total_charges = df_clean['account_Charges.Total'].median()
            df_clean['account_Charges.Total'] = df_clean['account_Charges.Total'].fillna(median_total_charges)


    binary_cols_before_map = ['customer_Partner', 'customer_Dependents', 'phone_PhoneService', 'account_PaperlessBilling']
    for col in binary_cols_before_map:
        if col in df_clean.columns and df_clean[col].isnull().any():

            df_clean[col] = df_clean[col].fillna('Unknown_Binary')


    if 'Churn' in df_clean.columns and df_clean['Churn'].isnull().any():

         df_clean['Churn'] = df_clean['Churn'].fillna(-1)


    return df_clean

def remove_duplicates(df):
    df_dedup = df.copy()

    if 'customerID' in df_dedup.columns:
        df_dedup = df_dedup.drop_duplicates(subset=['customerID'], keep='last')
    else:

        df_dedup = df_dedup.drop_duplicates()

    return df_dedup

def fix_categorical_inconsistencies(df):
    df_consistent = df.copy()


    corrections = {
        'customer_gender': {'M': 'Male', 'F': 'Female', 'male': 'Male', 'female': 'Female'},
        'customer_Partner': {'Y': 'Yes', 'N': 'No', 'yes': 'Yes', 'no': 'No', 'Unknown_Binary': 'No'},
        'customer_Dependents': {'Y': 'Yes', 'N': 'No', 'yes': 'Yes', 'no': 'No', 'Unknown_Binary': 'No'},
        'phone_PhoneService': {'Y': 'Yes', 'N': 'No', 'yes': 'Yes', 'no': 'No', 'No phone service': 'No', 'Unknown_Binary': 'No'},
        'account_PaperlessBilling': {'Y': 'Yes', 'N': 'No', 'yes': 'Yes', 'no': 'No', 'Unknown_Binary': 'No'},

    }

    for col, mapping in corrections.items():
        if col in df_consistent.columns:

            if df_consistent[col].dtype in ['object', 'category']:
                 df_consistent[col] = df_consistent[col].astype(str).replace(mapping)
            else:
                pass


    return df_consistent

def add_daily_charges(df):
    df_new = df.copy()

    if 'account_Charges.Monthly' in df_new.columns:

        df_new['account_Charges.Monthly'] = pd.to_numeric(df_new['account_Charges.Monthly'], errors='coerce')

        df_new['Contas_Diarias'] = df_new['account_Charges.Monthly'] / 30
        df_new['Contas_Diarias'] = df_new['Contas_Diarias'].fillna(0)
    else:

        df_new['Contas_Diarias'] = 0.0

    return df_new

def transform_binary_columns(df):
    df_transformed = df.copy()

    binary_map = {'Yes': 1, 'No': 0, 'No phone service': 0}



    binary_columns = ['customer_Partner', 'customer_Dependents', 'phone_PhoneService', 'account_PaperlessBilling', 'Churn']

    for col in binary_columns:
        if col in df_transformed.columns:
            if df_transformed[col].dtype not in ['int64', 'float64']:
                df_transformed[col] = df_transformed[col].map(binary_map)
            if df_transformed[col].isnull().any():
                 print(f"Warning: NaN values introduced in {col} after binary mapping. Consider updating binary_map.")


    return df_transformed

def standardize_data_types(df):
    df_typed = df.copy()


    categorical_columns = ['customer_gender', 'internet_InternetService', 'account_Contract', 'account_PaymentMethod',
                           'phone_MultipleLines', 'internet_OnlineSecurity', 'internet_OnlineBackup', 'internet_DeviceProtection',
                           'internet_TechSupport', 'internet_StreamingTV', 'internet_StreamingMovies']
    for col in categorical_columns:
        if col in df_typed.columns:
            df_typed[col] = df_typed[col].astype(str).astype('category')


    integer_columns = ['customer_tenure', 'customer_SeniorCitizen',
                       'customer_Partner', 'customer_Dependents',
                       'phone_PhoneService', 'account_PaperlessBilling', 'Churn']

    for col in ['account_Charges.Monthly', 'account_Charges.Total', 'Contas_Diarias']:
         if col in df_typed.columns:
              df_typed[col] = pd.to_numeric(df_typed[col], errors='coerce')

    for col in integer_columns:
        if col in df_typed.columns:
            if df_typed[col].isnull().any():
                print(f"Warning: NaN values found in column '{col}' before converting to integer. Filling with -1.")
                df_typed[col] = df_typed[col].fillna(-1)
            df_typed[col] = df_typed[col].astype('int64')
    float_columns = ['account_Charges.Monthly', 'account_Charges.Total', 'Contas_Diarias']
    for col in float_columns:
        if col in df_typed.columns:
            df_typed[col] = df_typed[col].astype('float64')

    return df_typed

def calculate_descriptive_statistics(df):
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns

    stats_summary = {}
    stats_summary['numeric'] = df[numeric_cols].describe()
    stats_summary['categorical'] = df[categorical_cols].describe()

    return stats_summary

def analyze_churn_distribution(df):
    if 'Churn' not in df.columns:
        return None

    if df['Churn'].dtype != 'category':
        churn_series = df['Churn'].astype('category')
    else:
        churn_series = df['Churn']


    churn_counts = churn_series.value_counts()
    churn_percentages = churn_series.value_counts(normalize=True) * 100

    distribution_summary = {
        'counts': churn_counts,
        'percentages': churn_percentages
    }

    return distribution_summary

def analyze_categorical_vs_churn(df, categorical_columns):
    if 'Churn' not in df.columns:
        return None

    results = {}

    for col in categorical_columns:
        if col in df.columns and df[col].dtype in ['object', 'category']:
            if df['Churn'].dtype != 'category':
                 churn_series = df['Churn'].astype('category')
            else:
                 churn_series = df['Churn']

            crosstab = pd.crosstab(df[col], churn_series, normalize='index') * 100
            results[col] = crosstab

    return results

def analyze_numerical_vs_churn(df, numerical_columns):
    if 'Churn' not in df.columns:
        return None

    results = {}

    for col in numerical_columns:
        if col in df.columns and df[col].dtype in [np.number]:
            if df['Churn'].dtype != np.number:
                 churn_series = df['Churn'].astype(np.number)
            else:
                 churn_series = df['Churn']

            group_stats = df.groupby(churn_series)[col].agg(['mean', 'median', 'std', 'count'])
            results[col] = group_stats

    return results


def create_churn_visualizations(df):
    visualizations_data = {}

    if 'Churn' in df.columns:
        if df['Churn'].dtype != 'category':
            churn_series = df['Churn'].astype('category')
        else:
            churn_series = df['Churn']

        churn_counts = churn_series.value_counts()
        visualizations_data['churn_distribution'] = {
            'labels': churn_counts.index.tolist(),
            'values': churn_counts.values.tolist()
        }

    return visualizations_data

def generate_correlation_matrix(df):
    numeric_df = df.select_dtypes(include=[np.number])
    correlation_matrix = numeric_df.corr()
    return correlation_matrix

def generate_introduction():
    introduction = {
        'objective': 'Analisar o comportamento de churn dos clientes da Telecom X',
        'problem': 'Alto índice de cancelamentos impacta a receita e crescimento da empresa',
        'goal': 'Identificar padrões e fatores que levam à evasão para desenvolver estratégias de retenção'
    }
    return introduction

def summarize_data_cleaning(original_records, final_records, issues_fixed):
    cleaning_summary = {
        'original_records': original_records,
        'final_records': final_records,
        'retention_rate': (final_records / original_records) * 100 if original_records > 0 else 0,
        'issues_addressed': issues_fixed
    }
    return cleaning_summary

def extract_key_insights(analysis_results):
    insights = []

    if 'churn_distribution' in analysis_results and analysis_results['churn_distribution'] is not None:
        churn_percentages = analysis_results['churn_distribution'].get('percentages')
        if isinstance(churn_percentages, pd.Series) and 1.0 in churn_percentages.index:
            churn_rate = churn_percentages[1.0]
            insights.append(f"Taxa geral de churn: {churn_rate:.1f}%")
        else:
            insights.append("Insight sobre taxa geral de churn não disponível devido a dados insuficientes ou formato inesperado.")


    if 'categorical_analysis' in analysis_results and analysis_results['categorical_analysis'] is not None:
        contract_analysis = analysis_results['categorical_analysis'].get('account_Contract', None)
        if contract_analysis is not None:
            if 'Month-to-month' in contract_analysis.index and 1.0 in contract_analysis.columns:
                 monthly_churn = contract_analysis.loc['Month-to-month', 1.0]
                 insights.append(f"Contratos mensais apresentam maior risco de churn: {monthly_churn:.1f}%")
            else:
                 insights.append("Insight sobre contratos mensais não disponível devido a dados insuficientes ou formato inesperado.")


    if 'numerical_analysis' in analysis_results and analysis_results['numerical_analysis'] is not None:
        tenure_analysis = analysis_results['numerical_analysis'].get('customer_tenure', None)
        if tenure_analysis is not None:
            if 1.0 in tenure_analysis.index and 0.0 in tenure_analysis.index:
                 avg_tenure_churned = tenure_analysis.loc[1.0, 'mean']
                 avg_tenure_retained = tenure_analysis.loc[0.0, 'mean']
                 insights.append(f"Clientes que cancelaram têm tenure médio de {avg_tenure_churned:.1f} meses vs {avg_tenure_retained:.1f} meses dos que permaneceram")
            else:
                 insights.append("Insight sobre tenure médio não disponível devido a dados insuficientes ou formato inesperado.")

    return insights

def generate_recommendations():
    recommendations = [
        'Implementar programa de fidelização para contratos de longo prazo',
        'Melhorar processo de onboarding para novos clientes',
        'Incentivar métodos de pagamento automáticos com descontos',
        'Criar campanhas direcionadas para clientes de alto risco',
        'Desenvolver sistema de alertas para identificação precoce de churn',
        'Oferecer benefícios escalonados baseados no tempo de permanência'
    ]
    return recommendations

def print_formatted_report(report):
    print("RELATÓRIO DE ANÁLISE DE CHURN - TELECOM X")
    print("="*50)

    print("\n1. INTRODUÇÃO")
    print("-"*20)
    intro = report.get('introducao', {})
    print(f"Objetivo: {intro.get('objective', 'N/A')}")
    print(f"Problema: {intro.get('problem', 'N/A')}")
    print(f"Meta: {intro.get('goal', 'N/A')}")

    print("\n2. LIMPEZA E TRATAMENTO DE DADOS")
    print("-"*35)
    cleaning = report.get('limpeza_dados', {})
    print(f"Registros originais: {cleaning.get('original_records', 'N/A')}")
    print(f"Registros finais: {cleaning.get('final_records', 'N/A')}")
    retention_rate = cleaning.get('retention_rate', 'N/A')
    if isinstance(retention_rate, (int, float)):
        print(f"Taxa de retenção: {retention_rate:.1f}%")
    else:
        print(f"Taxa de retenção: {retention_rate}")

    print("Correções aplicadas:")
    issues = cleaning.get('issues_addressed', [])
    if issues:
        for issue in issues:
            print(f"  - {issue}")
    else:
        print("  Nenhuma informação disponível.")

    print("\n3. ANÁLISE EXPLORATÓRIA DE DADOS")
    print("-"*32)
    eda = report.get('analise_exploratoria', {})
    print(f"Método: {eda.get('metodo', 'N/A')}")
    print(f"Ferramentas: {eda.get('ferramentas', 'N/A')}")
    vars_analyzed = eda.get('variaveis_analisadas', [])
    print(f"Variáveis analisadas: {', '.join(vars_analyzed)}" if vars_analyzed else "N/A")

    print("\n4. PRINCIPAIS INSIGHTS")
    print("-"*22)
    insights = report.get('insights', [])
    if insights:
        for insight in insights:
            print(f"  - {insight}")
    else:
        print("  Nenhum insight disponível.")

    print("\n5. CONCLUSÕES")
    print("-"*14)
    conclusions = report.get('conclusoes', [])
    if conclusions:
        for conclusion in conclusions:
            print(f"  - {conclusion}")
    else:
        print("  Nenhuma conclusão disponível.")


    print("\n6. RECOMENDAÇÕES")
    print("-"*17)
    recommendations = report.get('recomendacoes', [])
    if recommendations:
        for recommendation in recommendations:
            print(f"  - {recommendation}")
    else:
        print("  Nenhuma recomendação disponível.")

In [ ]:
def standardize_data_types(df):
    df_typed = df.copy()

    categorical_columns = ['customer_gender', 'internet_InternetService', 'account_Contract', 'account_PaymentMethod',
                           'phone_MultipleLines', 'internet_OnlineSecurity', 'internet_OnlineBackup', 'internet_DeviceProtection',
                           'internet_TechSupport', 'internet_StreamingTV', 'internet_StreamingMovies']
    for col in categorical_columns:
        if col in df_typed.columns:
            df_typed[col] = df_typed[col].astype(str).astype('category')


    integer_columns = ['customer_tenure', 'customer_SeniorCitizen',
                       'customer_Partner', 'customer_Dependents',
                       'phone_PhoneService', 'account_PaperlessBilling', 'Churn']

    for col in ['account_Charges.Monthly', 'account_Charges.Total', 'Contas_Diarias']:
         if col in df_typed.columns:
              df_typed[col] = pd.to_numeric(df_typed[col], errors='coerce')

    for col in integer_columns:
        if col in df_typed.columns:
            if df_typed[col].isnull().any():
                print(f"Warning: NaN values found in column '{col}' before converting to integer. Filling with -1.")
                df_typed[col] = df_typed[col].fillna(-1)
            df_typed[col] = df_typed[col].astype('int64')
    float_columns = ['account_Charges.Monthly', 'account_Charges.Total', 'Contas_Diarias']
    for col in float_columns:
        if col in df_typed.columns:
            df_typed[col] = df_typed[col].astype('float64')

    return df_typed

In [ ]:

df_raw = extract_data()

if df_raw is not None:
    df_transformed = transform_data(df_raw)

    analysis_results = analyze_data(df_transformed)

    final_report = generate_report(df_raw, df_transformed, analysis_results)
    print_formatted_report(final_report)

    X, y = prepare_data_for_modeling(df_transformed)

    print('\nProporção de clientes (após preparação para modelagem):')
    churn_ratio = y.value_counts(normalize=True) * 100
    print(churn_ratio.rename({0.0: 'Ativos', 1.0: 'Churn'}).round(1), '%\n')



    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=0.30,
        stratify=y,
        random_state=42
    )
    print(f"Data split into training ({len(X_train)} samples) and testing ({len(X_test)} samples) sets.")



    model = train_model(X_train, y_train)


    evaluation_metrics = evaluate_model(model, X_test, y_test)


    print("\n7. AVALIAÇÃO DO MODELO")
    print("-"*25)
    print(f"Accuracy: {evaluation_metrics['accuracy']:.4f}")
    print(f'Precision: {evaluation_metrics["precision"]:.4f}')
    print(f'Recall: {evaluation_metrics["recall"]:.4f}')
    print(f'F1 Score: {evaluation_metrics["f1_score"]:.4f}')
    print(f'AUC-ROC: {evaluation_metrics["roc_auc"]:.4f}')

else:
    print("Data extraction failed. Skipping subsequent steps.")

Conectando à API da Telecom X...
7267 registros extraídos com sucesso!

Iniciando transformação dos dados...
Transformação concluída.

Iniciando análise exploratória...


/usr/local/lib/python3.12/dist-packages/pandas/core/dtypes/common.py:1645: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  npdtype = np.dtype(dtype)
/usr/local/lib/python3.12/dist-packages/pandas/core/dtypes/common.py:1645: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  npdtype = np.dtype(dtype)
/usr/local/lib/python3.12/dist-packages/pandas/core/dtypes/common.py:1645: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  npdtype = np.dtype(dtype)


Análise exploratória concluída.

Gerando relatório...
Relatório gerado.
RELATÓRIO DE ANÁLISE DE CHURN - TELECOM X

1. INTRODUÇÃO
--------------------
Objetivo: Analisar o comportamento de churn dos clientes da Telecom X
Problema: Alto índice de cancelamentos impacta a receita e crescimento da empresa
Meta: Identificar padrões e fatores que levam à evasão para desenvolver estratégias de retenção

2. LIMPEZA E TRATAMENTO DE DADOS
-----------------------------------
Registros originais: 7267
Registros finais: 7267
Taxa de retenção: 100.0%
Correções aplicadas:
  - Valores ausentes tratados
  - Duplicatas removidas
  - Tipos padronizados
  - Inconsistências corrigidas

3. ANÁLISE EXPLORATÓRIA DE DADOS
--------------------------------
Método: Análise descritiva, distribuição de churn, análise categórica e numérica
Ferramentas: Python, Pandas, visualizações estatísticas
Variáveis analisadas: gender, tenure, Contract, MonthlyCharges, TotalCharges, PaymentMethod, InternetService

4. PRINCIPAIS 

ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [ ]:
def evaluate_model(model, X_test, y_test):
    """
    Evaluates the trained model.

    Args:
        model (RandomForestClassifier): The trained model.
        X_test (pd.DataFrame): The testing features.
        y_test (pd.Series): The testing target.

    Returns:
        dict: A dictionary containing the evaluation metrics.
    """
    print("\nAvaliando modelo...")
    # Ensure no NaNs in testing data before predicting and evaluating
    test_nan_mask = y_test.isna()

    if test_nan_mask.any():
        X_test_cleaned = X_test[~test_nan_mask].copy() # Use copy to avoid SettingWithCopyWarning
        y_test_cleaned = y_test[~test_nan_mask].copy() # Use copy to avoid SettingWithCopyWarning
    else:
        X_test_cleaned = X_test.copy() # Use copy
        y_test_cleaned = y_test.copy() # Use copy

    # Filter out samples where the true churn value is -1 (or any value not 0 or 1)
    # as these are likely placeholders for missing values introduced during transformation
    valid_indices = y_test_cleaned.isin([0, 1])
    X_test_cleaned = X_test_cleaned[valid_indices]
    y_test_cleaned = y_test_cleaned[valid_indices]


    y_pred = model.predict(X_test_cleaned)

    # Ensure y_test_cleaned and y_pred are of the correct type for metrics (integer)
    y_test_cleaned = y_test_cleaned.astype(int)
    y_pred = y_pred.astype(int)


    accuracy = accuracy_score(y_test_cleaned, y_pred)
    precision = precision_score(y_test_cleaned, y_pred)
    recall = recall_score(y_test_cleaned, y_pred)
    f1 = f1_score(y_test_cleaned, y_pred)
    # AUC-ROC for binary classification requires probability predictions
    # Get probabilities for the positive class (class 1)
    y_pred_proba = model.predict_proba(X_test_cleaned)[:, 1]
    roc_auc = roc_auc_score(y_test_cleaned, y_pred_proba) # Use probabilities for AUC


    metrics = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'roc_auc': roc_auc
    }
    print("Avaliação concluída.")
    return metrics

# Re-run the evaluation step in the main execution cell

In [ ]:
df_raw = extract_data()

if df_raw is not None:

    df_transformed = transform_data(df_raw)


    analysis_results = analyze_data(df_transformed)


    final_report = generate_report(df_raw, df_transformed, analysis_results)
    print_formatted_report(final_report)

    X, y = prepare_data_for_modeling(df_transformed)


    print('\nProporção de clientes (após preparação para modelagem):')
    churn_ratio = y.value_counts(normalize=True) * 100
    print(churn_ratio.rename({0.0: 'Ativos', 1.0: 'Churn'}).round(1), '%\n')



    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=0.30,
        stratify=y,
        random_state=42
    )
    print(f"Data split into training ({len(X_train)} samples) and testing ({len(X_test)} samples) sets.")



    model = train_model(X_train, y_train)


    evaluation_metrics = evaluate_model(model, X_test, y_test)

    print("\n7. AVALIAÇÃO DO MODELO")
    print("-"*25)
    print(f"Accuracy: {evaluation_metrics['accuracy']:.4f}")
    print(f'Precision: {evaluation_metrics["precision"]:.4f}')
    print(f'Recall: {evaluation_metrics["recall"]:.4f}')
    print(f'F1 Score: {evaluation_metrics["f1_score"]:.4f}')
    print(f'AUC-ROC: {evaluation_metrics["roc_auc"]:.4f}')

else:
    print("Data extraction failed. Skipping subsequent steps.")

Conectando à API da Telecom X...
7267 registros extraídos com sucesso!

Iniciando transformação dos dados...
Transformação concluída.

Iniciando análise exploratória...


/usr/local/lib/python3.12/dist-packages/pandas/core/dtypes/common.py:1645: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  npdtype = np.dtype(dtype)
/usr/local/lib/python3.12/dist-packages/pandas/core/dtypes/common.py:1645: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  npdtype = np.dtype(dtype)
/usr/local/lib/python3.12/dist-packages/pandas/core/dtypes/common.py:1645: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  npdtype = np.dtype(dtype)


Análise exploratória concluída.

Gerando relatório...
Relatório gerado.
RELATÓRIO DE ANÁLISE DE CHURN - TELECOM X

1. INTRODUÇÃO
--------------------
Objetivo: Analisar o comportamento de churn dos clientes da Telecom X
Problema: Alto índice de cancelamentos impacta a receita e crescimento da empresa
Meta: Identificar padrões e fatores que levam à evasão para desenvolver estratégias de retenção

2. LIMPEZA E TRATAMENTO DE DADOS
-----------------------------------
Registros originais: 7267
Registros finais: 7267
Taxa de retenção: 100.0%
Correções aplicadas:
  - Valores ausentes tratados
  - Duplicatas removidas
  - Tipos padronizados
  - Inconsistências corrigidas

3. ANÁLISE EXPLORATÓRIA DE DADOS
--------------------------------
Método: Análise descritiva, distribuição de churn, análise categórica e numérica
Ferramentas: Python, Pandas, visualizações estatísticas
Variáveis analisadas: gender, tenure, Contract, MonthlyCharges, TotalCharges, PaymentMethod, InternetService

4. PRINCIPAIS 

ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [ ]:
def evaluate_model(model, X_test, y_test):
    """
    Evaluates the trained model.

    Args:
        model (RandomForestClassifier): The trained model.
        X_test (pd.DataFrame): The testing features.
        y_test (pd.Series): The testing target.

    Returns:
        dict: A dictionary containing the evaluation metrics.
    """
    print("\nAvaliando modelo...")


    combined_test = pd.concat([X_test, y_test], axis=1)
    combined_test_cleaned = combined_test.dropna(subset=[y_test.name]).copy()


    X_test_cleaned = combined_test_cleaned.drop(columns=[y_test.name])
    y_test_cleaned = combined_test_cleaned[y_test.name]


    valid_indices = y_test_cleaned.isin([0, 1])
    X_test_cleaned = X_test_cleaned[valid_indices].copy()
    y_test_cleaned = y_test_cleaned[valid_indices].copy()


    y_test_cleaned = y_test_cleaned.astype(int)



    y_pred = model.predict(X_test_cleaned)

    y_pred = y_pred.astype(int)



    accuracy = accuracy_score(y_test_cleaned, y_pred)
    precision = precision_score(y_test_cleaned, y_pred, pos_label=1)
    recall = recall_score(y_test_cleaned, y_pred, pos_label=1)
    f1 = f1_score(y_test_cleaned, y_pred, pos_label=1)

e
    if hasattr(model, 'predict_proba'):
        y_pred_proba = model.predict_proba(X_test_cleaned)[:, 1]
        roc_auc = roc_auc_score(y_test_cleaned, y_pred_proba)
    else:

        roc_auc = np.nan


    metrics = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'roc_auc': roc_auc
    }
    print("Avaliação concluída.")
    return metrics

## Update cell content



In [ ]:
def train_model(X_train, y_train):
    """
    Trains a RandomForestClassifier model.

    Args:
        X_train (pd.DataFrame): The training features.
        y_train (pd.Series): The training target.

    Returns:
        RandomForestClassifier: The trained model.
    """
    print("\nTreinando modelo...")
    valid_train_indices = y_train.isin([0, 1])
    X_train_cleaned = X_train[valid_train_indices].copy()
    y_train_cleaned = y_train[valid_train_indices].copy()

    model = RandomForestClassifier(random_state=42)
    model.fit(X_train_cleaned, y_train_cleaned)
    print("Modelo treinado.")
    return model

def evaluate_model(model, X_test, y_test):
    """
    Evaluates the trained model.

    Args:
        model (RandomForestClassifier): The trained model.
        X_test (pd.DataFrame): The testing features.
        y_test (pd.Series): The testing target.

    Returns:
        dict: A dictionary containing the evaluation metrics.
    """
    print("\nAvaliando modelo...")


    combined_test = pd.concat([X_test, y_test], axis=1)

    combined_test_cleaned = combined_test.dropna(subset=[y_test.name]).copy()


    valid_indices = combined_test_cleaned[y_test.name].isin([0, 1])
    combined_test_cleaned = combined_test_cleaned[valid_indices].copy()

    X_test_cleaned = combined_test_cleaned.drop(columns=[y_test.name])
    y_test_cleaned = combined_test_cleaned[y_test.name]

    y_test_cleaned = y_test_cleaned.astype(int)

    y_pred = model.predict(X_test_cleaned)


    y_pred = y_pred.astype(int)

      accuracy = accuracy_score(y_test_cleaned, y_pred)
    precision = precision_score(y_test_cleaned, y_pred, pos_label=1, zero_division=0)
    recall = recall_score(y_test_cleaned, y_pred, pos_label=1, zero_division=0)
    f1 = f1_score(y_test_cleaned, y_pred, pos_label=1, zero_division=0)


    if hasattr(model, 'predict_proba') and len(y_test_cleaned.unique()) > 1:
        y_pred_proba = model.predict_proba(X_test_cleaned)[:, 1]
        roc_auc = roc_auc_score(y_test_cleaned, y_pred_proba)
    else:
        print("Warning: Cannot calculate AUC-ROC. Model lacks predict_proba or test set is not binary.")
        roc_auc = np.nan

    metrics = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'roc_auc': roc_auc
    }
    print("Avaliação concluída.")
    return metrics



In [ ]:

df_raw = extract_data()
if df_raw is not None:
    df_transformed = transform_data(df_raw)
    analysis_results = analyze_data(df_transformed)
    final_report = generate_report(df_raw, df_transformed, analysis_results)
    print_formatted_report(final_report)

    X, y = prepare_data_for_modeling(df_transformed)
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=0.30,
        stratify=y,
        random_state=42
    )


    model = train_model(X_train, y_train)
    evaluation_metrics = evaluate_model(model, X_test, y_test)

    print("\n7. AVALIAÇÃO DO MODELO")
    print("-"*25)
    print(f"Accuracy: {evaluation_metrics['accuracy']:.4f}")
    print(f"Precision: {evaluation_metrics['precision']:.4f}")
    print(f"Recall: {evaluation_metrics['recall']:.4f}")
    print(f"F1 Score: {evaluation_metrics['f1_score']:.4f}")

    if not np.isnan(evaluation_metrics['roc_auc']):
        print(f'AUC-ROC: {evaluation_metrics["roc_auc"]:.4f}')
    else:
        print('AUC-ROC: N/A (Model lacks predict_proba or test set not binary)')


Conectando à API da Telecom X...
7267 registros extraídos com sucesso!

Iniciando transformação dos dados...
Transformação concluída.

Iniciando análise exploratória...


/usr/local/lib/python3.12/dist-packages/pandas/core/dtypes/common.py:1645: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  npdtype = np.dtype(dtype)
/usr/local/lib/python3.12/dist-packages/pandas/core/dtypes/common.py:1645: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  npdtype = np.dtype(dtype)
/usr/local/lib/python3.12/dist-packages/pandas/core/dtypes/common.py:1645: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  npdtype = np.dtype(dtype)


Análise exploratória concluída.

Gerando relatório...
Relatório gerado.
RELATÓRIO DE ANÁLISE DE CHURN - TELECOM X

1. INTRODUÇÃO
--------------------
Objetivo: Analisar o comportamento de churn dos clientes da Telecom X
Problema: Alto índice de cancelamentos impacta a receita e crescimento da empresa
Meta: Identificar padrões e fatores que levam à evasão para desenvolver estratégias de retenção

2. LIMPEZA E TRATAMENTO DE DADOS
-----------------------------------
Registros originais: 7267
Registros finais: 7267
Taxa de retenção: 100.0%
Correções aplicadas:
  - Valores ausentes tratados
  - Duplicatas removidas
  - Tipos padronizados
  - Inconsistências corrigidas

3. ANÁLISE EXPLORATÓRIA DE DADOS
--------------------------------
Método: Análise descritiva, distribuição de churn, análise categórica e numérica
Ferramentas: Python, Pandas, visualizações estatísticas
Variáveis analisadas: gender, tenure, Contract, MonthlyCharges, TotalCharges, PaymentMethod, InternetService

4. PRINCIPAIS 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
from datetime import datetime
import warnings
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

warnings.filterwarnings('ignore')


def extract_data(url="https://raw.githubusercontent.com/ingridcristh/challenge2-data-science/main/TelecomX_Data.json"):
    """
    Extracts data from a given URL.

    Args:
        url (str): The URL to extract data from.

    Returns:
        pd.DataFrame: The extracted data as a pandas DataFrame, or None if an error occurs.
    """
    print("Conectando à API da Telecom X...")
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
            'Accept': 'application/json'
        }
        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()
        data = response.json()
        if isinstance(data, list):
            df = pd.DataFrame(data)
        else:
            df = pd.DataFrame(data.get('customers', data))
        print(f"{len(df)} registros extraídos com sucesso!")
        return df
    except requests.exceptions.RequestException as e:
        print(f"Erro na requisição: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"Erro ao decodificar JSON: {e}")
        return None

def transform_data(df):
    """
    Applies a series of transformations to the raw data.

    Args:
        df (pd.DataFrame): The raw DataFrame.

    Returns:
        pd.DataFrame: The transformed DataFrame.
    """
    print("\nIniciando transformação dos dados...")
    df_flat = flatten_nested_columns(df)
    df_clean = handle_missing_values(df_flat)
    df_dedup = remove_duplicates(df_clean)
    df_consistent = fix_categorical_inconsistencies(df_dedup)
    df_daily = add_daily_charges(df_consistent)
    df_binary = transform_binary_columns(df_daily)
    df_final = standardize_data_types(df_binary)
    print("Transformação concluída.")
    return df_final

def analyze_data(df):
    """
    Performs exploratory data analysis on the transformed data.

    Args:
        df (pd.DataFrame): The transformed DataFrame.

    Returns:
        dict: A dictionary containing the analysis results.
    """
    print("\nIniciando análise exploratória...")
    descriptive_stats = calculate_descriptive_statistics(df)
    churn_distribution = analyze_churn_distribution(df)

    categorical_cols = [col for col in df.select_dtypes(include=['object', 'category']).columns if col != 'Churn']
    categorical_analysis = analyze_categorical_vs_churn(df, categorical_cols)

    numerical_cols = [col for col in df.select_dtypes(include=[np.number]).columns if col not in ['customer_SeniorCitizen', 'Churn', 'customer_Partner', 'customer_Dependents', 'phone_PhoneService', 'account_PaperlessBilling']]
    numerical_analysis = analyze_numerical_vs_churn(df, numerical_cols)
    correlations = generate_correlation_matrix(df)
    visualizations = create_churn_visualizations(df)
    analysis_results = {
        'descriptive_statistics': descriptive_stats,
        'churn_distribution': churn_distribution,
        'categorical_analysis': categorical_analysis,
        'numerical_analysis': numerical_analysis,
        'correlations': correlations,
        'visualizations': visualizations
    }
    print("Análise exploratória concluída.")
    return analysis_results

def generate_report(df_original, df_final, analysis_results):
    """
    Generates a final report based on the analysis results.

    Args:
        df_original (pd.DataFrame): The original DataFrame.
        df_final (pd.DataFrame): The final transformed DataFrame.
        analysis_results (dict): The results from the data analysis.

    Returns:
        dict: A dictionary containing the report sections.
    """
    print("\nGerando relatório...")
    report = {}
    report['introducao'] = generate_introduction()
    report['limpeza_dados'] = summarize_data_cleaning(
        len(df_original),
        len(df_final),
        ['Valores ausentes tratados', 'Duplicatas removidas', 'Tipos padronizados', 'Inconsistências corrigidas']
    )
    report['analise_exploratoria'] = {
        'metodo': 'Análise descritiva, distribuição de churn, análise categórica e numérica',
        'ferramentas': 'Python, Pandas, visualizações estatísticas',
        'variaveis_analisadas': ['customer_gender', 'customer_tenure', 'account_Contract', 'account_Charges.Monthly', 'account_Charges.Total', 'account_PaymentMethod', 'internet_InternetService']
    }
    report['insights'] = extract_key_insights(analysis_results)
    report['conclusoes'] = [
        'Contratos mensais representam maior risco de churn',
        'Clientes novos são mais propensos ao cancelamento',
        'Método de pagamento influencia na retenção',
        'Tempo de permanência é inversamente correlacionado ao churn',
        'Serviço de internet e tipo de contrato são fortes preditores de churn.'
    ]
    report['recomendacoes'] = generate_recommendations()
    print("Relatório gerado.")
    return report

def prepare_data_for_modeling(df):
    """
    Prepares the data for machine learning modeling.

    Args:
        df (pd.DataFrame): The transformed DataFrame.

    Returns:
        tuple: A tuple containing the features (X) and target (y) DataFrames.
    """
    print("\nPreparando dados para modelagem...")
    df_cleaned = df[df['Churn'].isin([0, 1])].copy()

    if 'customerID' in df_cleaned.columns:
        df_cleaned = df_cleaned.drop(columns=['customerID'])

    y = df_cleaned['Churn']
    X = df_cleaned.drop(columns=['Churn'])

    columns_to_drop = ['customer', 'phone', 'internet', 'account', 'Contas_Diarias']
    X = X.drop(columns=[col for col in columns_to_drop if col in X.columns])


    cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()

    if cat_cols:
        encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop='first')
        encoded = encoder.fit_transform(X[cat_cols])
        encoded_df = pd.DataFrame(
            encoded,
            columns=encoder.get_feature_names_out(cat_cols),
            index=X.index
        )
        X = pd.concat([X.drop(columns=cat_cols), encoded_df], axis=1)

    num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
    if num_cols:
        scaler = StandardScaler()
        X[num_cols] = scaler.fit_transform(X[num_cols])

    print("Dados preparados para modelagem.")
    return X, y

def train_model(X_train, y_train):
    """
    Trains a RandomForestClassifier model.

    Args:
        X_train (pd.DataFrame): The training features.
        y_train (pd.Series): The training target.

    Returns:
        RandomForestClassifier: The trained model.
    """
    print("\nTreinando modelo...")

    valid_train_indices = y_train.isin([0, 1])
    X_train_cleaned = X_train[valid_train_indices].copy()
    y_train_cleaned = y_train[valid_train_indices].copy()

    model = RandomForestClassifier(random_state=42)
    model.fit(X_train_cleaned, y_train_cleaned)
    print("Modelo treinado.")
    return model

def evaluate_model(model, X_test, y_test):
    """
    Evaluates the trained model.

    Args:
        model (RandomForestClassifier): The trained model.
        X_test (pd.DataFrame): The testing features.
        y_test (pd.Series): The testing target.

    Returns:
        dict: A dictionary containing the evaluation metrics.
    """
    print("\nAvaliando modelo...")

    combined_test = pd.concat([X_test, y_test], axis=1)


    valid_indices = combined_test[y_test.name].isin([0, 1])
    combined_test_cleaned = combined_test[valid_indices].copy()

    X_test_cleaned = combined_test_cleaned.drop(columns=[y_test.name])
    y_test_cleaned = combined_test_cleaned[y_test.name]

    y_test_cleaned = y_test_cleaned.astype(int)

    y_pred = model.predict(X_test_cleaned)

    y_pred = y_pred.astype(int)

    accuracy = accuracy_score(y_test_cleaned, y_pred)
    precision = precision_score(y_test_cleaned, y_pred, pos_label=1, zero_division=0)
    recall = recall_score(y_test_cleaned, y_pred, pos_label=1, zero_division=0)
    f1 = f1_score(y_test_cleaned, y_pred, pos_label=1, zero_division=0)

    if hasattr(model, 'predict_proba') and len(y_test_cleaned.unique()) > 1 and 0 in y_test_cleaned.unique() and 1 in y_test_cleaned.unique():
        y_pred_proba = model.predict_proba(X_test_cleaned)[:, 1]
        roc_auc = roc_auc_score(y_test_cleaned, y_pred_proba)
    else:
        print("Warning: Cannot calculate AUC-ROC. Model lacks predict_proba or test set is not strictly binary (contains only one class of 0 or 1, or other values).")
        roc_auc = np.nan


    metrics = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'roc_auc': roc_auc
    }
    print("Avaliação concluída.")
    return metrics




def flatten_nested_columns(df):
    df_flat = df.copy()
    nested_cols = ['customer', 'phone', 'internet', 'account']
    for col in nested_cols:
        if col in df_flat.columns:
            flattened_data = pd.json_normalize(df_flat[col].apply(lambda x: x if isinstance(x, dict) else {}))
            flattened_data.columns = [f"{col}_{sub_col}" for sub_col in flattened_data.columns]
            df_flat = pd.concat([df_flat.drop(columns=[col]), flattened_data], axis=1)
    return df_flat

def handle_missing_values(df):
    df_clean = df.copy()

    if 'customer_gender' in df_clean.columns and df_clean['customer_gender'].isnull().any():
        df_clean['customer_gender'] = df_clean['customer_gender'].astype(str).replace('nan', np.nan)
        mode_gender = df_clean['customer_gender'].mode()
        if not mode_gender.empty:
            df_clean['customer_gender'] = df_clean['customer_gender'].fillna(mode_gender[0])
        else:
            df_clean['customer_gender'] = df_clean['customer_gender'].fillna('Unknown')


    if 'internet_InternetService' in df_clean.columns and df_clean['internet_InternetService'].isnull().any():
        df_clean['internet_InternetService'] = df_clean['internet_InternetService'].fillna('No internet service')

    if 'account_Charges.Total' in df_clean.columns:
        df_clean['account_Charges.Total'] = pd.to_numeric(df_clean['account_Charges.Total'], errors='coerce')
        mask_new_customers = (df_clean['account_Charges.Total'].isnull()) & (df_clean['customer_tenure'] <= 1)
        if 'account_Charges.Monthly' in df_clean.columns:
             df_clean['account_Charges.Monthly'] = pd.to_numeric(df_clean['account_Charges.Monthly'], errors='coerce')
             df_clean.loc[mask_new_customers, 'account_Charges.Total'] = df_clean.loc[mask_new_customers, 'account_Charges.Monthly']

        remaining_nulls = df_clean['account_Charges.Total'].isnull()
        if remaining_nulls.any():
            median_total_charges = df_clean['account_Charges.Total'].median()
            df_clean['account_Charges.Total'] = df_clean['account_Charges.Total'].fillna(median_total_charges)


    binary_cols_before_map = ['customer_Partner', 'customer_Dependents', 'phone_PhoneService', 'account_PaperlessBilling']
    for col in binary_cols_before_map:
        if col in df_clean.columns and df_clean[col].isnull().any():
            df_clean[col] = df_clean[col].fillna('Unknown_Binary')

    if 'Churn' in df_clean.columns and df_clean['Churn'].isnull().any():
         df_clean['Churn'] = df_clean['Churn'].fillna(-1)


    return df_clean

def remove_duplicates(df):
    df_dedup = df.copy()
    if 'customerID' in df_dedup.columns:
        df_dedup = df_dedup.drop_duplicates(subset=['customerID'], keep='last')
    else:
        df_dedup = df_dedup.drop_duplicates()

    return df_dedup

def fix_categorical_inconsistencies(df):
    df_consistent = df.copy()
    corrections = {
        'customer_gender': {'M': 'Male', 'F': 'Female', 'male': 'Male', 'female': 'Female'},
        'customer_Partner': {'Y': 'Yes', 'N': 'No', 'yes': 'Yes', 'no': 'No', 'Unknown_Binary': 'No'},
        'customer_Dependents': {'Y': 'Yes', 'N': 'No', 'yes': 'Yes', 'no': 'No', 'Unknown_Binary': 'No'},
        'phone_PhoneService': {'Y': 'Yes', 'N': 'No', 'yes': 'Yes', 'no': 'No', 'No phone service': 'No', 'Unknown_Binary': 'No'},
        'account_PaperlessBilling': {'Y': 'Yes', 'N': 'No', 'yes': 'Yes', 'no': 'No', 'Unknown_Binary': 'No'},
    }

    for col, mapping in corrections.items():
        if col in df_consistent.columns:
            df_consistent[col] = df_consistent[col].astype(str).replace(mapping)


    return df_consistent

def add_daily_charges(df):
    df_new = df.copy()
    if 'account_Charges.Monthly' in df_new.columns:
        df_new['account_Charges.Monthly'] = pd.to_numeric(df_new['account_Charges.Monthly'], errors='coerce')
        df_new['Contas_Diarias'] = df_new['account_Charges.Monthly'] / 30
        df_new['Contas_Diarias'] = df_new['Contas_Diarias'].fillna(0)
    else:
        df_new['Contas_Diarias'] = 0.0

    return df_new

def transform_binary_columns(df):
    df_transformed = df.copy()
    binary_map = {'Yes': 1, 'No': 0}

    binary_columns = ['customer_Partner', 'customer_Dependents', 'phone_PhoneService', 'account_PaperlessBilling']

    for col in binary_columns:
        if col in df_transformed.columns:
            df_transformed[col] = df_transformed[col].astype(str).map(binary_map).fillna(0).astype(int)

    if 'Churn' in df_transformed.columns:
         df_transformed['Churn'] = df_transformed['Churn'].astype(str).map({'Yes': 1, 'No': 0}).fillna(df_transformed['Churn']).astype(int)


    return df_transformed


def standardize_data_types(df):
    df_typed = df.copy()


    categorical_columns = ['customer_gender', 'internet_InternetService', 'account_Contract', 'account_PaymentMethod',
                           'phone_MultipleLines', 'internet_OnlineSecurity', 'internet_OnlineBackup', 'internet_DeviceProtection',
                           'internet_TechSupport', 'internet_StreamingTV', 'internet_StreamingMovies']
    for col in categorical_columns:
        if col in df_typed.columns:

            df_typed[col] = df_typed[col].astype(str).astype('category')



    numeric_columns = {
        'customer_tenure': 'int64',
        'customer_SeniorCitizen': 'int64',
        'account_Charges.Monthly': 'float64',
        'account_Charges.Total': 'float64',
        'Contas_Diarias': 'float64'
        }


    for col, dtype in numeric_columns.items():
        if col in df_typed.columns:
            df_typed[col] = pd.to_numeric(df_typed[col], errors='coerce')
            if dtype == 'int64' and df_typed[col].isnull().any():
                 print(f"Warning: NaN values found in numerical column '{col}' after conversion. Filling with -1.")
                 df_typed[col] = df_typed[col].fillna(-1)
            elif dtype == 'float64' and df_typed[col].isnull().any():
                 print(f"Warning: NaN values found in float column '{col}' after conversion. Filling with median.")
                 df_typed[col] = df_typed[col].fillna(df_typed[col].median())


            try:
                df_typed[col] = df_typed[col].astype(dtype)
            except Exception as e:
                 print(f"Error converting column '{col}' to {dtype}: {e}")


    return df_typed

def calculate_descriptive_statistics(df):
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns

    stats_summary = {}
    if 'Churn' in df.columns and -1 in df['Churn'].unique():
         df_numeric_filtered = df[df['Churn'].isin([0, 1])]
    else:
         df_numeric_filtered = df

    stats_summary['numeric'] = df_numeric_filtered[numeric_cols].describe()
    stats_summary['categorical'] = df[categorical_cols].describe()

    return stats_summary

def analyze_churn_distribution(df):
    if 'Churn' not in df.columns:
        return None

    churn_series_filtered = df[df['Churn'].isin([0, 1])]['Churn']

    if churn_series_filtered.empty:
         print("Warning: No valid Churn values (0 or 1) found for distribution analysis.")
         return None

    if churn_series_filtered.dtype != 'category':
        churn_series_categorical = churn_series_filtered.astype('category')
    else:
        churn_series_categorical = churn_series_filtered


    churn_counts = churn_series_categorical.value_counts()
    churn_percentages = churn_series_categorical.value_counts(normalize=True) * 100

    distribution_summary = {
        'counts': churn_counts,
        'percentages': churn_percentages
    }

    return distribution_summary

def analyze_categorical_vs_churn(df, categorical_columns):
    if 'Churn' not in df.columns:
        return None

    results = {}


    df_filtered = df[df['Churn'].isin([0, 1])].copy()

    if df_filtered.empty:
        print("Warning: No valid Churn values (0 or 1) found for categorical analysis.")
        return None


    churn_series_filtered = df_filtered['Churn'].astype('category')


    for col in categorical_columns:
        if col in df_filtered.columns and df_filtered[col].dtype in ['object', 'category']:
            crosstab = pd.crosstab(df_filtered[col], churn_series_filtered, normalize='index') * 100
            results[col] = crosstab

    return results

def analyze_numerical_vs_churn(df, numerical_columns):
    if 'Churn' not in df.columns:
        return None

    results = {}

    df_filtered = df[df['Churn'].isin([0, 1])].copy()

    if df_filtered.empty:
         print("Warning: No valid Churn values (0 or 1) found for numerical analysis.")
         return None


    churn_series_filtered = df_filtered['Churn'].astype(np.number)

    for col in numerical_columns:
        if col in df_filtered.columns and df_filtered[col].dtype in [np.number]:
            group_stats = df_filtered.groupby(churn_series_filtered)[col].agg(['mean', 'median', 'std', 'count'])
            results[col] = group_stats

    return results


def create_churn_visualizations(df):
    visualizations_data = {}

    if 'Churn' in df.columns:
        churn_series_filtered = df[df['Churn'].isin([0, 1])]['Churn']

        if churn_series_filtered.empty:
             print("Warning: No valid Churn values (0 or 1) found for visualization data.")
             return visualizations_data

        if churn_series_filtered.dtype != 'category':
            churn_series_categorical = churn_series_filtered.astype('category')
        else:
            churn_series_categorical = churn_series_filtered


        churn_counts = churn_series_categorical.value_counts()
        visualizations_data['churn_distribution'] = {
            'labels': churn_counts.index.tolist(),
            'values': churn_counts.values.tolist()
        }
    return visualizations_data

def generate_correlation_matrix(df):
    if 'Churn' in df.columns and -1 in df['Churn'].unique():
         df_numeric = df[df['Churn'].isin([0, 1])].select_dtypes(include=[np.number])
    else:
         df_numeric = df.select_dtypes(include=[np.number])

    correlation_matrix = df_numeric.corr()
    return correlation_matrix

def generate_introduction():
    introduction = {
        'objective': 'Analisar o comportamento de churn dos clientes da Telecom X',
        'problem': 'Alto índice de cancelamentos impacta a receita e crescimento da empresa',
        'goal': 'Identificar padrões e fatores que levam à evasão para desenvolver estratégias de retenção'
    }
    return introduction

def summarize_data_cleaning(original_records, final_records, issues_fixed):
    cleaning_summary = {
        'original_records': original_records,
        'final_records': final_records,
        'retention_rate': (final_records / original_records) * 100 if original_records > 0 else 0,
        'issues_addressed': issues_fixed
    }
    return cleaning_summary

def extract_key_insights(analysis_results):
    insights = []

    if 'churn_distribution' in analysis_results and analysis_results['churn_distribution'] is not None:
        churn_percentages = analysis_results['churn_distribution'].get('percentages')
        if isinstance(churn_percentages, pd.Series) and 1 in churn_percentages.index:
             churn_rate = churn_percentages[1]
             insights.append(f"Taxa geral de churn: {churn_rate:.1f}%")
        else:
             insights.append("Taxa geral de churn: Não disponível (dados inválidos ou ausentes após filtragem).")


    if 'categorical_analysis' in analysis_results and analysis_results['categorical_analysis'] is not None:
        contract_analysis = analysis_results['categorical_analysis'].get('account_Contract', None)
        if contract_analysis is not None:

            if 'Month-to-month' in contract_analysis.index and 1 in contract_analysis.columns:
                 monthly_churn = contract_analysis.loc['Month-to-month', 1]
                 insights.append(f"Contratos mensais apresentam maior risco de churn: {monthly_churn:.1f}%")
            else:
                 insights.append("Insight sobre contratos mensais não disponível devido a dados insuficientes ou formato inesperado após filtragem.")


    if 'numerical_analysis' in analysis_results and analysis_results['numerical_analysis'] is not None:
        tenure_analysis = analysis_results['numerical_analysis'].get('customer_tenure', None)
        if tenure_analysis is not None:

            if 1 in tenure_analysis.index and 0 in tenure_analysis.index:
                 avg_tenure_churned = tenure_analysis.loc[1, 'mean']
                 avg_tenure_retained = tenure_analysis.loc[0, 'mean']
                 insights.append(f"Clientes que cancelaram têm tenure médio de {avg_tenure_churned:.1f} meses vs {avg_tenure_retained:.1f} meses dos que permaneceram")
            else:
                 insights.append("Insight sobre tenure médio não disponível devido a dados insuficientes ou formato inesperado após filtragem.")

    return insights

def generate_recommendations():
    recommendations = [
        'Implementar programa de fidelização para contratos de longo prazo',
        'Melhorar processo de onboarding para novos clientes',
        'Incentivar métodos de pagamento automáticos com descontos',
        'Criar campanhas direcionadas para clientes de alto risco',
        'Desenvolver sistema de alertas para identificação precoce de churn',
        'Oferecer benefícios escalonados baseados no tempo de permanência'
    ]
    return recommendations

def print_formatted_report(report):
    print("RELATÓRIO DE ANÁLISE DE CHURN - TELECOM X")
    print("="*50)

    print("\n1. INTRODUÇÃO")
    print("-"*20)
    intro = report.get('introducao', {})
    print(f"Objetivo: {intro.get('objective', 'N/A')}")
    print(f"Problema: {intro.get('problem', 'N/A')}")
    print(f"Meta: {intro.get('goal', 'N/A')}")

    print("\n2. LIMPEZA E TRATAMENTO DE DADOS")
    print("-"*35)
    cleaning = report.get('limpeza_dados', {})
    print(f"Registros originais: {cleaning.get('original_records', 'N/A')}")
    print(f"Registros finais: {cleaning.get('final_records', 'N/A')}")
    retention_rate = cleaning.get('retention_rate', 'N/A')
    if isinstance(retention_rate, (int, float)):
        print(f"Taxa de retenção: {retention_rate:.1f}%")
    else:
        print(f"Taxa de retenção: {retention_rate}")

    print("Correções aplicadas:")
    issues = cleaning.get('issues_addressed', [])
    if issues:
        for issue in issues:
            print(f"  - {issue}")
    else:
        print("  Nenhuma informação disponível.")

    print("\n3. ANÁLISE EXPLORATÓRIA DE DADOS")
    print("-"*32)
    eda = report.get('analise_exploratoria', {})
    print(f"Método: {eda.get('metodo', 'N/A')}")
    print(f"Ferramentas: {eda.get('ferramentas', 'N/A')}")
    vars_analyzed = eda.get('variaveis_analisadas', [])
    print(f"Variáveis analisadas: {', '.join(vars_analyzed)}" if vars_analyzed else "N/A")

    print("\n4. PRINCIPAIS INSIGHTS")
    print("-"*22)
    insights = report.get('insights', [])
    if insights:
        for insight in insights:
            print(f"  - {insight}")
    else:
        print("  Nenhum insight disponível.")

    print("\n5. CONCLUSÕES")
    print("-"*14)
    conclusions = report.get('conclusoes', [])
    if conclusions:
        for conclusion in conclusions:
            print(f"  - {conclusion}")
    else:
        print("  Nenhuma conclusão disponível.")


    print("\n6. RECOMENDAÇÕES")
    print("-"*17)
    recommendations = report.get('recomendacoes', [])
    if recommendations:
        for recommendation in recommendations:
            print(f"  - {recommendation}")
    else:
        print("  Nenhuma recomendação disponível.")


df_raw = extract_data()
if df_raw is not None:
    df_transformed = transform_data(df_raw)

    df_analyzed = df_transformed[df_transformed['Churn'].isin([0, 1])].copy()
    analysis_results = analyze_data(df_analyzed)
    final_report = generate_report(df_raw, df_analyzed, analysis_results)
    print_formatted_report(final_report)


    X, y = prepare_data_for_modeling(df_transformed)
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=0.30,
        stratify=y,
        random_state=42
    )

    model = train_model(X_train, y_train)
    evaluation_metrics = evaluate_model(model, X_test, y_test)

    print("\n7. AVALIAÇÃO DO MODELO")
    print("-"*25)
    print(f"Accuracy: {evaluation_metrics['accuracy']:.4f}")
    print(f"Precision: {evaluation_metrics['precision']:.4f}")
    print(f"Recall: {evaluation_metrics['recall']:.4f}")
    print(f"F1 Score: {evaluation_metrics['f1_score']:.4f}")
    if not np.isnan(evaluation_metrics['roc_auc']):
        print(f'AUC-ROC: {evaluation_metrics["roc_auc"]:.4f}')
    else:
        print('AUC-ROC: N/A (Model lacks predict_proba or test set not strictly binary)')
plt.figure(figsize=(12, 10))

if 'Churn' in df_transformed.columns and -1 in df_transformed['Churn'].unique():
     df_numeric_filtered_for_plot = df_transformed[df_transformed['Churn'].isin([0, 1])].select_dtypes(include=[np.number])
else:
     df_numeric_filtered_for_plot = df_transformed.select_dtypes(include=[np.number])

if 'Churn' in df_numeric_filtered_for_plot.columns:
    corr = df_numeric_filtered_for_plot.corr()
    sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm')
    plt.title('Matriz de Correlação (excluindo Churn=-1)')
    plt.tight_layout()
    plt.show()
else:
    print("Correlation matrix cannot be plotted as 'Churn' column is not suitable for numeric correlation after filtering.")


if 'Churn' in df_transformed.columns and -1 in df_transformed['Churn'].unique():
     df_filtered_for_plots = df_transformed[df_transformed['Churn'].isin([0, 1])].copy()
else:
     df_filtered_for_plots = df_transformed.copy()

if 'Churn' in df_filtered_for_plots.columns:
    df_filtered_for_plots['Churn_Label'] = df_filtered_for_plots['Churn'].map({0: 'Não', 1: 'Sim'})

    if 'customer_tenure' in df_filtered_for_plots.columns:
        plt.figure(figsize=(10, 4))
        sns.boxplot(x='Churn_Label', y='customer_tenure', data=df_filtered_for_plots)
        plt.title('Tempo de Contrato × Churn')
        plt.xlabel('Churn')
        plt.ylabel('Tempo de Contrato (meses)')
        plt.tight_layout()
        plt.show()
    else:
        print("Boxplot for Tenure vs Churn cannot be plotted as 'customer_tenure' column is missing.")

    if 'account_Charges.Total' in df_filtered_for_plots.columns:
        plt.figure(figsize=(10, 4))
        sns.boxplot(x='Churn_Label', y='account_Charges.Total', data=df_filtered_for_plots)
        plt.title('Total Gasto × Churn')
        plt.xlabel('Churn')
        plt.ylabel('Total Gasto')
        plt.tight_layout()
        plt.show()
    else:
         print("Boxplot for Total Charges vs Churn cannot be plotted as 'account_Charges.Total' column is missing.")
else:
    print("Boxplots for Churn analysis cannot be plotted as 'Churn' column is missing or not suitable after filtering.")


Conectando à API da Telecom X...
7267 registros extraídos com sucesso!

Iniciando transformação dos dados...


ValueError: invalid literal for int() with base 10: ''

In [ ]:
def transform_binary_columns(df):
    df_transformed = df.copy()
    binary_map = {'Yes': 1, 'No': 0}

    binary_columns = ['customer_Partner', 'customer_Dependents', 'phone_PhoneService', 'account_PaperlessBilling']

    for col in binary_columns:
        if col in df_transformed.columns:
            df_transformed[col] = df_transformed[col].astype(str).map(binary_map).fillna(0).astype(int)

    if 'Churn' in df_transformed.columns:
         df_transformed['_Churn_mapped'] = df_transformed['Churn'].astype(str).map({'Yes': 1, 'No': 0}).fillna(df_transformed['Churn'])

         df_transformed['Churn'] = pd.to_numeric(df_transformed['_Churn_mapped'], errors='coerce').fillna(-1).astype(int)

         df_transformed = df_transformed.drop(columns=['_Churn_mapped'])


    return df_transformed

df_raw = extract_data()
if df_raw is not None:
    df_transformed = transform_data(df_raw)
    df_analyzed = df_transformed[df_transformed['Churn'].isin([0, 1])].copy()
    analysis_results = analyze_data(df_analyzed)
    final_report = generate_report(df_raw, df_analyzed, analysis_results)
    print_formatted_report(final_report)

    X, y = prepare_data_for_modeling(df_transformed)
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=0.30,
        stratify=y,
        random_state=42
    )


    model = train_model(X_train, y_train)
    evaluation_metrics = evaluate_model(model, X_test, y_test)

    print("\n7. AVALIAÇÃO DO MODELO")
    print("-"*25)
    print(f"Accuracy: {evaluation_metrics['accuracy']:.4f}")
    print(f"Precision: {evaluation_metrics['precision']:.4f}")
    print(f"Recall: {evaluation_metrics['recall']:.4f}")
    print(f"F1 Score: {evaluation_metrics['f1_score']:.4f}")
    if not np.isnan(evaluation_metrics['roc_auc']):
        print(f'AUC-ROC: {evaluation_metrics["roc_auc"]:.4f}')
    else:
        print('AUC-ROC: N/A (Model lacks predict_proba or test set not strictly binary)')


Conectando à API da Telecom X...
7267 registros extraídos com sucesso!

Iniciando transformação dos dados...
Transformação concluída.

Iniciando análise exploratória...


/usr/local/lib/python3.12/dist-packages/pandas/core/dtypes/common.py:1645: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  npdtype = np.dtype(dtype)


Análise exploratória concluída.

Gerando relatório...
Relatório gerado.
RELATÓRIO DE ANÁLISE DE CHURN - TELECOM X

1. INTRODUÇÃO
--------------------
Objetivo: Analisar o comportamento de churn dos clientes da Telecom X
Problema: Alto índice de cancelamentos impacta a receita e crescimento da empresa
Meta: Identificar padrões e fatores que levam à evasão para desenvolver estratégias de retenção

2. LIMPEZA E TRATAMENTO DE DADOS
-----------------------------------
Registros originais: 7267
Registros finais: 7043
Taxa de retenção: 96.9%
Correções aplicadas:
  - Valores ausentes tratados
  - Duplicatas removidas
  - Tipos padronizados
  - Inconsistências corrigidas

3. ANÁLISE EXPLORATÓRIA DE DADOS
--------------------------------
Método: Análise descritiva, distribuição de churn, análise categórica e numérica
Ferramentas: Python, Pandas, visualizações estatísticas
Variáveis analisadas: customer_gender, customer_tenure, account_Contract, account_Charges.Monthly, account_Charges.Total, acc

In [ ]:
def print_formatted_report(report):
    print("RELATÓRIO DE ANÁLISE DE CHURN - TELECOM X")
    print("="*50)

    print("\n1. INTRODUÇÃO")
    print("-"*20)
    intro = report['introducao']
    print(f"Objetivo: {intro['objective']}")
    print(f"Problema: {intro['problem']}")
    print(f"Meta: {intro['goal']}")

    print("\n2. LIMPEZA E TRATAMENTO DE DADOS")
    print("-"*35)
    cleaning = report['limpeza_dados']
    print(f"Registros originais: {cleaning['original_records']}")
    print(f"Registros finais: {cleaning['final_records']}")
    print(f"Taxa de retenção: {cleaning['retention_rate']:.1f}%")
    print("Correções aplicadas:")
    for issue in cleaning['issues_addressed']:
        print(f"  - {issue}")

    print("\n3. ANÁLISE EXPLORATÓRIA DE DADOS")
    print("-"*32)
    eda = report['analise_exploratoria']
    print(f"Método: {eda['metodo']}")
    print(f"Ferramentas: {eda['ferramentas']}")
    print(f"Variáveis analisadas: {', '.join(eda['variaveis_analisadas'])}")

    print("\n4. PRINCIPAIS INSIGHTS")
    print("-"*22)
    for insight in report['insights']:
        print(f"  - {insight}")

    print("\n5. CONCLUSÕES")
    print("-"*14)
    for conclusion in report['conclusoes']:
        print(f"  - {conclusion}")

    print("\n6. RECOMENDAÇÕES")
    print("-"*17)
    for recommendation in report['recomendacoes']:
        print(f"  - {recommendation}")

def run_final_report_pipeline(df_original, df_final, analysis_results):
    final_report = create_final_report(df_original, df_final, analysis_results)
    print_formatted_report(final_report)
    return final_report